[1. Introduction](#1.-Introduction)

[2. Dataset Description](#2.-Dataset-Description)

[3. Data processing](#3.-Data-processing)
- [3.1 Missing data](#3.1-Missing-data)
- [3.2 Outliers](#3.2-Outliers)
- [3.3 Create features](#3.3-Create-features)

[4. Analysis](#4.-Analysis)
- [4.1 Temporal Analysis](#4.1-Temporal-Analysis)
- [4.2 Spatial Analysis](#4.2-Spatial-Analysis)
- [4.3 Temporal and Spatial Analysis](#4.3-Temporal-and-Spatial-Analysis)

[5. Clustering](#5.Clustering)
- [5.1 DBSCAN](#5.1-DBSCAN)
- [5.2 K-DBSCAN](#5.2-K-DBSCAN)

In [ ]:
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from pandas.api.types import CategoricalDtype
import folium
from folium.plugins import HeatMap
from datetime import datetime
import random
import altair as alt
import geopandas as gpd
import minisom
from sklearn import preprocessing, cluster
import scipy
import calendar
import time
from selenium import webdriver

from sklearn.neighbors import NearestNeighbors
from kneed import KneeLocator
from ipyleaflet import Map, ImageOverlay, basemap_to_tiles, basemaps
from sklearn.manifold import MDS
from sklearn.preprocessing import MinMaxScaler
# Import Sklearn DBSCAN algorithm
from sklearn.cluster import DBSCAN as dbscan
from sklearn.cluster import DBSCAN
# Import Sklearn OPTICS algorithm
from sklearn.cluster import OPTICS, cluster_optics_dbscan
import matplotlib.gridspec as gridspec
import matplotlib.dates as mdates
import math

from sklearn.preprocessing import StandardScaler


from vega_datasets import data
from scipy.cluster.vq import kmeans

url = 'https://github.com/python-visualization/folium/blob/master/tests/us-counties.json'

accidents_file = './input/Accident_Information.csv'
lad_file = './input/Local_Authority_District_to_Country__December_2017__Lookup_in_the_United_Kingdom.csv'
ward_file = './input/Ward_to_Local_Authority_District_to_County_to_Region_to_Country__December_2017__Lookup_in_United_Kingdom_version_2.csv'

# Set parameters for number of min/max rows

#pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 50)

import warnings
warnings.filterwarnings('ignore')

from geopy.distance import great_circle


# Perceptually color space
from colorspacious import cspace_convert
import matplotlib.colors as col

# Ticker
import matplotlib.ticker as ticker
import matplotlib as mpl

# Colormap
import branca
import branca.colormap as cm

# Axes 3D
from mpl_toolkits.mplot3d import Axes3D


# Multipoint
from shapely.geometry import MultiPoint

# KDBSCAN
import kdbscan
from kdbscan import KDBSCAN

# define the number of kilometers in one radian
kms_per_radian = 6371.0088
spatial_dist_max = 20 / kms_per_radian
temporal_dist_max = 7

In [ ]:
# mean shift clustering
from numpy import unique
from numpy import where
from sklearn.datasets import make_classification
from sklearn.cluster import MeanShift

### Read shape files for geospatial analysis

In [ ]:
uk_region = pd.read_csv('input/uk_regions_data.csv')
uk_region = uk_region.append({'Area_Code': 'E09000001', 'Region': 'London', 'Area': 'City of London'}, ignore_index=True)


#gb = gpd.read_file('shapefiles/boundaries_gb.shp')
gb = gpd.read_file('shapefiles/new/Local_Authority_Districts__May_2020__Boundaries_UK_BFE.shp')
gb.rename(columns={'lad20cd': 'geo_code'},inplace=True)

gb.crs = 'epsg:27700'
gb.loc[gb['geo_code']=='E41000052','geo_code'] = 'E06000052'
gb.loc[gb['geo_code']=='E41000324','geo_code'] = 'E09000033'

## Define auxiliary functions

In [ ]:
def hist_date(df, column_name='start_date', color='#494949', title=''):
    """
    """
    plt.figure(figsize=(20, 10))
    ax = (df[column_name].groupby(df[column_name].dt.hour)
                         .count()).plot(kind="bar", color=color)
    ax.set_facecolor('#eeeeee')
    ax.set_xlabel("hour of the day")
    ax.set_ylabel("count")
    ax.set_title(title)
    plt.show()

### Functions to handle spatial and spatio-temporal distances

In [ ]:
def great_circle2(lat1, long1, lat2, long2):

    # Convert latitude and longitude to 
    # spherical coordinates in radians.
    degrees_to_radians = math.pi/180.0
        
    # phi = 90 - latitude
    phi1 = (90.0 - lat1)*degrees_to_radians
    phi2 = (90.0 - lat2)*degrees_to_radians
        
    # theta = longitude
    theta1 = long1*degrees_to_radians
    theta2 = long2*degrees_to_radians
    
    # Compute spherical distance from spherical coordinates.
        
    # For two locations in spherical coordinates 
    # (1, theta, phi) and (1, theta, phi)
    # cosine( arc length ) = 
    #    sin phi sin phi' cos(theta-theta') + cos phi cos phi'
    # distance = rho * arc length
    
    cos = (math.sin(phi1)*math.sin(phi2)*math.cos(theta1 - theta2) + 
           math.cos(phi1)*math.cos(phi2))
    
    if (cos > 1.0):
        cos = 1.0

    arc = math.acos( cos )

    # Remember to multiply arc by the radius of the earth 
    # in your favorite set of units to get length.
    return arc

def SpaceDistance(x,y):
    try:
        gc_dist = great_circle2(x[1],x[0],y[1],y[0])
    except ValueError:
        gc_dist = np.Infinity
    
    if (gc_dist>spatial_dist_max):
        return np.Infinity
    else:
        return gc_dist
    #return great_circle(x[1],x[0],y[1],y[0])

def SpaceTimeDistance(x,y,diff_time,spatial_dist_max,temporal_dist_max):
    #print('Params = {},{}'.format(spatial_dist_max,temporal_dist_max))
    diff_time = math.fabs(x[2] - y[2])
    if (np.isnan(diff_time) or diff_time > temporal_dist_max):
        return np.Infinity
    
    try:
        gc_dist = great_circle2(x[1],x[0],y[1],y[0])
    except ValueError:
        #print(x[1],x[0],y[1],y[0])
        gc_dist = np.Infinity
    
    if (gc_dist>spatial_dist_max):
        return np.Infinity
    
    ratio_t=diff_time/temporal_dist_max
    ratio_d=gc_dist/spatial_dist_max
    if (ratio_d>ratio_t):
        return gc_dist
    else:
        return ratio_t * spatial_dist_max

In [ ]:
def getColor (x, y, minX, maxX, minY, maxY):
    wX=maxX-minX 
    wY=maxY-minY
    rr=y-minY 
    cc=x-minX
    #print(x,y)
    if (wY < wX):   #scale vertically, i.e. modify rr
        rr *= wX/wY  
    else:           #scale horizontally, i.e. modify cc
        cc *= wY/wX
    maxD=max(wX,wY)
    rr1=maxD-rr
    cc1=maxD-cc
    #print(rr,cc,maxD,rr1,cc1)
    dc=[math.sqrt(rr*rr+cc*cc),math.sqrt(rr*rr+cc1*cc1),math.sqrt(rr1*rr1+cc*cc),math.sqrt(rr1*rr1+cc1*cc1)]
    weights=[0.0,0.0,0.0,0.0]
    for i in range(len(weights)):
        weights[i]=(maxD-dc[i])/maxD
        if (weights[i]<0):
            weights[i]=0
    #print(dc,weights)
    reds=[228,25,255,37]
    greens=[220,228,18,13]
    blues=[0,218,6,252]
    dr=0
    dg=0
    db=0
    for i,weight in enumerate(weights):
        dr += weight*reds[i]
        dg += weight*greens[i]
        db += weight*blues[i]
    if (dr<0):
        dr=0;
    if (dr>255):
        dr=255
    if (dg<0):
        dg=0;
    if (dg>255):
        dg=255        
    if (db<0):
        db=0;
    if (db>255):
        db=255  
    #print(weights,dr,dg,db)
    c_string = '#{:02x}{:02x}{:02x}'.format(int(dr),int(dg),int(db))    
    return c_string

In [ ]:
#df[df.Region=='London'][['Local_Authority_(District)','Area_Code']].value_counts()

In [ ]:
def df_group_altair(df,year):
    
    df_alt = df[df.Year==year]
    df_alt = df_alt.groupby(['Area_Code','Accident_Severity'])[['Accident_Index','Number_of_Casualties']].agg(
                        {'Accident_Index':'count','Number_of_Casualties':'sum'})
    df_alt = df_alt.unstack()
    df_alt = df_alt.assign(Casualties = (df_alt[('Number_of_Casualties',   'Slight')] + 
                                           df_alt[('Number_of_Casualties',   'Serious')] + 
                                           df_alt[('Number_of_Casualties',   'Fatal')]))
    df_alt.drop(columns=[('Number_of_Casualties',   'Slight'),
                          ('Number_of_Casualties',   'Serious'),
                          ('Number_of_Casualties',   'Fatal')],inplace=True,errors='ignore')
    df_alt['Year'] = year
    return df_alt

In [ ]:
def distplot_fig(data, x, hue=None, row=None, col=None, legend=True, hist=False, **kwargs):
    """A figure-level distribution plot with support for hue, col, row arguments."""
    bins = kwargs.pop('bins', None)
    if (bins is None) and hist: 
        # Make sure that the groups have equal-sized bins
        bins = np.histogram_bin_edges(data[x].dropna())
    g = sns.FacetGrid(data, hue=hue, row=row, col=col)
    g.map(sns.distplot, x, bins=bins, hist=hist, **kwargs)
    if legend and (hue is not None) and (hue not in [x, row, col]):
        g.add_legend(title=hue) 
    return g   

## Load datasets

In [ ]:
#pd.set_option('display.max_rows', 500)
#pd.reset_option('display.max_rows')

In [ ]:
df = pd.read_csv(accidents_file)

In [ ]:
df_lad = pd.read_csv(lad_file)

In [ ]:
df_ward = pd.read_csv(ward_file)
df_ward.drop_duplicates(subset=['LAD17NM','LAD17CD','CTRY17NM'],inplace=True)
df_ward = df_ward[['LAD17NM','LAD17CD','CTRY17NM']]

### Data processing

##### Check files

In [ ]:
df_ward.head(1)

In [ ]:
df.head(1)

#### Merge accidents and LAD dataset

In [ ]:
df = pd.merge(df, df_lad[['LAD17NM','CTRY17NM','LAD17CD']], how='left', 
              left_on='Local_Authority_(District)', right_on='LAD17NM')

df.drop(columns=['LAD17NM'],inplace=True,errors='ignore')

#### Merge accidents and Ward dataset

In [ ]:
df = pd.merge(df, df_ward, how='left', 
              left_on='Local_Authority_(Highway)', right_on='LAD17NM')

df.drop(columns=['LAD17NM'],inplace=True,errors='ignore')

In [ ]:
df['CTRY17NM_z'] = df['CTRY17NM_x'].where(df['CTRY17NM_x'].notnull(), df['CTRY17NM_y'])
df['LAD17CD_z'] = df['LAD17CD_x'].where(df['LAD17CD_x'].notnull(), df['LAD17CD_y'])

In [ ]:
df.CTRY17NM_z.isnull().groupby(df['Year']).sum().astype(int).reset_index(name='count')

In [ ]:
df[df.CTRY17NM_z.isna()==True]['Local_Authority_(District)'].value_counts()

##### Manual imputation for Local Authority District

In [ ]:
df['CTRY17NM_z'] =  np.where(
             df['Local_Authority_(District)']=='Edinburgh, City of', 
            'Scotland', 
             np.where(
                df['Local_Authority_(District)']=='Rhondda, Cynon, Taff','Wales', 
             np.where(
                df['Local_Authority_(District)']=='St. Albans','England', 
             np.where(
                df['Local_Authority_(District)']=='Stratford-upon-Avon','England',  
             np.where(
                df['Local_Authority_(District)']=='St. Edmundsbury','England', 
             np.where(
                df['Local_Authority_(District)']=='The Vale of Glamorgan','Wales', 
             np.where(
                df['Local_Authority_(District)']=='London Airport (Heathrow)','England', 
             np.where(
                df['Local_Authority_(District)']=='Western Isles','Scotland', df['CTRY17NM_z']     
             ))))))))

df['LAD17CD_z'] =  np.where(
             df['Local_Authority_(District)']=='Edinburgh, City of', 
            'S12000036', 
             np.where(
                df['Local_Authority_(District)']=='Rhondda, Cynon, Taff','W06000016', 
             np.where(
                df['Local_Authority_(District)']=='St. Albans','E07000240', 
             np.where(
                df['Local_Authority_(District)']=='Stratford-upon-Avon','E07000221',  
             np.where(
                df['Local_Authority_(District)']=='St. Edmundsbury','E07000204', 
             np.where(
                df['Local_Authority_(District)']=='The Vale of Glamorgan','W06000014', 
             np.where(
                df['Local_Authority_(District)']=='London Airport (Heathrow)','E09000017', 
             np.where(
                df['Local_Authority_(District)']=='Western Isles','S12000013', df['LAD17CD_z']     
             ))))))))

df.drop(columns=['LAD17CD_x','LAD17CD_y'],inplace=True,errors='ignore')
df.rename(columns={'LAD17CD_z': 'Area_Code'}, inplace=True)
df['Area_Code'] = np.where(df['Local_Authority_(District)']=='St. Albans',
                                               'E07000100',df['Area_Code'])
df['Area_Code'] = np.where(df['Local_Authority_(District)']=='East Hertfordshire',
                                               'E07000097',df['Area_Code'])
df['Area_Code'] = np.where(df['Local_Authority_(District)']=='Welwyn Hatfield',
                                               'E07000104',df['Area_Code'])

In [ ]:
df[df.CTRY17NM_z.isna()==True]['Local_Authority_(District)'].value_counts()

In [ ]:
df.drop(columns=['CTRY17NM_x','CTRY17NM_y','InScotland'],inplace=True,errors='ignore')

In [ ]:
df.rename(columns={'CTRY17NM_z': 'Country'}, inplace=True)

#### Merge accidents and Ward dataset

In [ ]:
df.shape

In [ ]:
df = pd.merge(df, uk_region[['Area_Code','Region']], how='left', 
              left_on='Area_Code', right_on='Area_Code')
#df.drop(columns=['LAD17NM'],inplace=True,errors='ignore')

## Summary Statistics

In [ ]:
df.head(3)

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
print('Date range',df.Date.min(),'until',df.Date.max())

In [ ]:
df_lad.head(1)

# 3. Data Processing

### 3.1 Missing data

In [ ]:
df.isna().sum()

We will now remove some variables form the dataset since they will not be useful for the analysis

In [ ]:
missing_df = df.isnull().sum(axis=0).reset_index()
missing_df.columns = ['column_name','missing_count']
missing_df = missing_df[missing_df['missing_count']>0]
missing_df = missing_df.sort_values(by='missing_count')

ind = np.arange(missing_df.shape[0])
width = 0.5
fig,ax = plt.subplots(figsize=(12,18))
rects = ax.barh(ind,missing_df.missing_count.values,color='blue')
ax.set_yticks(ind)
ax.set_yticklabels(missing_df.column_name.values, rotation='horizontal')
ax.set_xlabel("Count of missing values")
ax.set_title("Number of missing values in each column")
plt.show()

##### Drop columns

In [ ]:
drop_col = ['Junction_Control', 'LSOA_of_Accident_Location', '2nd_Road_Class', '2nd_Road_Number',
           'Pedestrian_Crossing-Human_Control', 'Pedestrian_Crossing-Physical_Facilities',
            'Did_Police_Officer_Attend_Scene_of_Accident','1st_Road_Number'] 

In [ ]:
df = df.drop(columns=drop_col, axis=1, errors='ignore')

### Drop rows

##### Missing values - Time

In [ ]:
df[df.Time.isna()==True].head(2)

In [ ]:
df.dropna(subset=['Time'],inplace=True)

##### Missing values - Latitude/Longitude

In [ ]:
df[df.Latitude.isna()==True].head(2)
df[df.Longitude.isna()==True].head(2)

In [ ]:
df.dropna(subset=['Latitude','Longitude'], inplace=True)

##### Missing values - Speed limit

In [ ]:
df[df.Speed_limit.isna()==True].head(2)
df.dropna(subset=['Speed_limit'], inplace=True)

##### Final check for missing data

In [ ]:
missing_df = df.isnull().sum(axis=0).reset_index()
missing_df.columns = ['column_name','missing_count']
missing_df = missing_df[missing_df['missing_count']>0]
missing_df = missing_df.sort_values(by='missing_count')

ind = np.arange(missing_df.shape[0])
width = 0.5
fig,ax = plt.subplots(figsize=(12,18))
rects = ax.barh(ind,missing_df.missing_count.values,color='blue')
ax.set_yticks(ind)
ax.set_yticklabels(missing_df.column_name.values, rotation='horizontal')
ax.set_xlabel("Count of missing values")
ax.set_title("Number of missing values in each column")
plt.show()

In [ ]:
df['Region'] =  np.where(df.Region.isna()==True,
             np.where(
                df['Local_Authority_(Highway)']=='Gateshead','North East', 
             np.where(
                df['Local_Authority_(Highway)']=='Northumberland','North East', 
             np.where(
                df['Local_Authority_(Highway)']=='Hertfordshire','East', 
             np.where(
                df['Local_Authority_(Highway)']=='Isles of Scilly','South West',  
             np.where(
                df['Local_Authority_(Highway)']=='City of London','London', 
             df['Region']     
             ))))),df['Region'])

In [ ]:
df.shape

### 3.3 Create features

Conversion of coordinates from string to float

In [ ]:
# df['Start_Lat'] = pd.to_numeric(df["Start_Lat"], downcast="float")
# df['Start_Lng'] = pd.to_numeric(df["Start_Lng"], downcast="float")

In [ ]:
df.head(1)

Conversion from string date to date format

In [ ]:
df['Datetime'] = pd.to_datetime(df['Date'] + ' ' + df['Time'])
df['Hour'] = df['Datetime'].dt.hour
df['WeekYear'] = df['Datetime'].dt.isocalendar().week
df['YearMonth'] = df['Datetime'].dt.to_period('M')
df['DaysSince'] = np.where(df.Year>=2013,(df['Datetime'] - pd.to_datetime('2013-01-01')).dt.days,0)
df['Month'] = df['Datetime'].dt.month.apply(lambda x: calendar.month_abbr[x])
df['DayYear'] = df['Datetime'].dt.dayofyear

df['Year_bin'] = np.where((df.Year >= 2006) & (df.Year <= 2008),'2006-2008',
                     np.where(
                        (df.Year >= 2009) & (df.Year <= 2011),'2009-2011', 
                     np.where(
                        (df.Year >= 2012) & (df.Year <= 2014),'2012-2014', 
                     np.where(
                         (df.Year >= 2015) & (df.Year <= 2017),'2015-2017',df.Year
                     ))))

df['Hour_bin'] = np.where((df.Hour >= 0) & (df.Hour < 4),'0-4h',
                     np.where(
                        (df.Hour >= 4) & (df.Hour < 8),'4-8h', 
                     np.where(
                        (df.Hour >= 8) & (df.Hour < 12),'8-12h', 
                     np.where(
                         (df.Hour >= 12) & (df.Hour < 16),'12-16h',
                     np.where(
                        (df.Hour >= 16) & (df.Hour < 20),'16-20h', 
                     np.where(
                        (df.Hour >= 20) & (df.Hour < 24),'20-24h', df.Hour
                     ))))))

### Exploratory Analysis

In [ ]:
plt.figure(figsize=(14,6))
plt.hist(df['Accident_Severity'])
plt.show()

### Accidents per LA 

In [ ]:
df_top15_lad = df[df['Local_Authority_(District)'].isin(df['Local_Authority_(District)'].value_counts()[:15].index)]

plt.figure(figsize=(14,4))
chart = sns.countplot(x=df_top15_lad['Local_Authority_(District)'],data=df_top15_lad,order = df_top15_lad['Local_Authority_(District)'].value_counts().index)
chart.set_xticklabels(chart.get_xticklabels(), rotation=45)
plt.show()

### Accidents per LA and Year bin (5 groups of 3 years each)

In [ ]:
df1la = df[df.Year_bin=='2006-2008']
df2la = df[df.Year_bin=='2009-2011']
df3la = df[df.Year_bin=='2012-2014']
df4la = df[df.Year_bin=='2015-2017']

df1_top15 = df1la[df1la['Local_Authority_(District)'].isin(df1la['Local_Authority_(District)'].value_counts()[:15].index)]
df2_top15 = df2la[df2la['Local_Authority_(District)'].isin(df2la['Local_Authority_(District)'].value_counts()[:15].index)]
df3_top15 = df3la[df3la['Local_Authority_(District)'].isin(df3la['Local_Authority_(District)'].value_counts()[:15].index)]
df4_top15 = df4la[df4la['Local_Authority_(District)'].isin(df4la['Local_Authority_(District)'].value_counts()[:15].index)]

In [ ]:


fig, ax = plt.subplots(2,2,figsize=(16,8))

sns.countplot(x=df1_top15['Local_Authority_(District)'],data=df1_top15,
              order = df1_top15['Local_Authority_(District)'].value_counts().index,ax=ax[0][0])
sns.countplot(x=df2_top15['Local_Authority_(District)'],
                      data=df2_top15,
                      order = df2_top15['Local_Authority_(District)'].value_counts().index,
                      ax=ax[0][1])
sns.countplot(x=df3_top15['Local_Authority_(District)'],
                      data=df3_top15,
                      order = df3_top15['Local_Authority_(District)'].value_counts().index,
                      ax=ax[1][0])
sns.countplot(x=df4_top15['Local_Authority_(District)'],
                      data=df4_top15,
                      order = df4_top15['Local_Authority_(District)'].value_counts().index,
                      ax=ax[1][1])

ax[0][0].set_xticklabels(ax[0][0].get_xticklabels(), rotation=90)
ax[0][1].set_xticklabels(ax[0][1].get_xticklabels(), rotation=90)
ax[1][0].set_xticklabels(ax[1][0].get_xticklabels(), rotation=90)
ax[1][1].set_xticklabels(ax[1][1].get_xticklabels(), rotation=90)

ax[0][0].set_title('Accidents per LA (2006-2008)\n')
ax[0][1].set_title('Accidents per LA (2009-2011)\n')
ax[1][0].set_title('Accidents per LA (2012-2014)\n')
ax[1][1].set_title('Accidents per LA (2015-2017)\n')

fig.tight_layout()
fig.show()

### Accidents per Region and Year bin (5 groups of 3 years each)

In [ ]:

df1_top15 = df1la[df1la['Region'].isin(df1la['Region'].value_counts()[:15].index)]
df2_top15 = df2la[df2la['Region'].isin(df2la['Region'].value_counts()[:15].index)]
df3_top15 = df3la[df3la['Region'].isin(df3la['Region'].value_counts()[:15].index)]
df4_top15 = df4la[df4la['Region'].isin(df4la['Region'].value_counts()[:15].index)]


fig, ax = plt.subplots(2,2,figsize=(16,12),sharey=True)

sns.countplot(x=df1_top15['Region'],data=df1_top15,
              order = df1_top15['Region'].value_counts().index,ax=ax[0][0])
sns.countplot(x=df2_top15['Region'],
                      data=df2_top15,
                      order = df2_top15['Region'].value_counts().index,
                      ax=ax[0][1])
sns.countplot(x=df3_top15['Region'],
                      data=df3_top15,
                      order = df3_top15['Region'].value_counts().index,
                      ax=ax[1][0])
sns.countplot(x=df4_top15['Region'],
                      data=df4_top15,
                      order = df4_top15['Region'].value_counts().index,
                      ax=ax[1][1])

ax[0][0].set_xticklabels(ax[0][0].get_xticklabels(), rotation=90)
ax[0][1].set_xticklabels(ax[0][1].get_xticklabels(), rotation=90)
ax[1][0].set_xticklabels(ax[1][0].get_xticklabels(), rotation=90)
ax[1][1].set_xticklabels(ax[1][1].get_xticklabels(), rotation=90)

ax[0][0].set_title('Accidents per Region (2006-2008)\n')
ax[0][1].set_title('Accidents per Region (2009-2011)\n')
ax[1][0].set_title('Accidents per Region (2012-2014)\n')
ax[1][1].set_title('Accidents per Region (2015-2017)\n')

ax[0][0].set_xlabel('')
ax[0][1].set_xlabel('')
ax[1][0].set_xlabel('')
ax[1][1].set_xlabel('')

fig.tight_layout()
fig.show()



### Number_of_Casualties (Distribution)

In [ ]:
plt.figure(figsize=(14,4))
sns.countplot(df.Number_of_Casualties)
plt.show()

#### Number_of_Casualties per Year bin

##### All casualties

In [ ]:

#sns.countplot(df.Number_of_Casualties)

grid = sns.FacetGrid(df[df.Year>2005], col='Year_bin',height=4,aspect=1.1)

grid.map(sns.countplot, 'Number_of_Casualties')

plt.show()

##### Only multiple casualties

In [ ]:

#sns.countplot(df.Number_of_Casualties)

grid = sns.FacetGrid(df[df.Number_of_Casualties > 1 & (df.Year>2005)], col='Year_bin', height=7, aspect=0.8)

grid.map(sns.countplot, 'Number_of_Casualties')

plt.show()

##### Outlier Analysis

In [ ]:
df.Number_of_Casualties.value_counts()

In [ ]:
df[df.Number_of_Casualties.isin([87,93])]

93 Casualties on 2014-10-20 in Hertfordshire:
* The accident happened in a double decker bus and despite the severity of the accident it was not fatal.    
* Reference: https://www.bbc.co.uk/news/uk-england-beds-bucks-herts-35416869
    
87 Casualties on 2014-06-03 in County Durham:
* The accident happened between two buses of children and it was not fatal. The reported casualties number was supposedly lower than the one from the dataset at around 50 casualties.
* Reference: https://www.chroniclelive.co.uk/news/north-east-news/stanley-bus-crash-view-images-7207590

87 Casualties on 2011-06-20	in Salford:
* The accident happened between two buses of children and it was not fatal. The reported casualties number was supposedly lower than the one from the dataset at around 50 casualties.
* Reference: https://www.lancashiretelegraph.co.uk/news/9095178.north-west-motorway-tailbacks-school-coach-crash/

##### Number_of_Vehicles (Distribution)

In [ ]:
plt.figure(figsize=(14,4))
sns.countplot(df.Number_of_Vehicles)
plt.show()

In [ ]:
df[df.Number_of_Vehicles > 30]

67 vehicles accidents with 70 casualties on 2013-09-05 in Kent:
Reference: https://www.theguardian.com/uk-news/2013/sep/05/car-pileup-sheppey-bridge-kent
    
37 vehicles accidents with 36 casualties on 2015-02-14 in Kent:
Reference:
    
34 vehicles accidents with 51 casualties on 2011-11-04 in Kent:
Reference:
    
32 vehicles accidents with 5 casualties on 2009-12-23 in Kent:
Reference:

In [ ]:
a = df[df.Year>=2010]

## Temporal Analysis

In [ ]:
a = df[df.Year>=2015]

In [ ]:
#a.groupby(['Year','Day_of_Week'])['Accident_Index'].count().reset_index()

In [ ]:
df[df.Year>=2015].groupby(['Date'])['Accident_Index'].count().reset_index()

In [ ]:
a

In [ ]:
a = df[(df.Year>=2010) & (df.Accident_Severity.isin(['Fatal','Serious']))].groupby(['Year','Month'])['Accident_Index'].count().reset_index()

df2010 = a[a.Year==2010]
df2011 = a[a.Year==2011]
df2012 = a[a.Year==2012]
df2013 = a[a.Year==2013]
df2014 = a[a.Year==2014]
df2015 = a[a.Year==2015]
df2016 = a[a.Year==2016]
df2017 = a[a.Year==2017]

dfall = a.groupby(['Month'])['Accident_Index'].sum().reset_index()

In [ ]:

df2010.plot.bar('Month','Accident_Index')
df2011.plot.bar(x='Month',y='Accident_Index')
df2012.plot.bar(x='Month',y='Accident_Index')
df2013.plot.bar(x='Month',y='Accident_Index')
df2014.plot.bar(x='Month',y='Accident_Index')
df2015.plot.bar(x='Month',y='Accident_Index')
df2016.plot.bar(x='Month',y='Accident_Index')
df2017.plot.bar(x='Month',y='Accident_Index')

dfall.plot.bar(x='Month',y='Accident_Index')


In [ ]:
df.head(1)

##### Accidents per year

In [ ]:
df.groupby(df['Year'])['Accident_Index'].count().reset_index()

In [ ]:
df_date = df.groupby(df.Date)['Accident_Index'].count().reset_index()
df_date = df_date.set_index('Date')
#df [(df.index > '2016-02-08') & (df.index < '2016-03-08')]

In [ ]:
df_pairplot = df[(df.Date>='2005-01-01') & (df.Date<='2005-02-01')]
df_pairplot = df_pairplot.groupby([df_pairplot.Date, df_pairplot.Accident_Severity,
                                  df_pairplot.Hour, df_pairplot.DaysSince,
                                  df_pairplot.Year])['Accident_Index','Number_of_Casualties'].agg(
                                 {'Accident_Index':'count', 'Number_of_Casualties':'sum'}).reset_index()
df_pairplot

In [ ]:
print(df1la[df1la.Region=='West Midlands']['Accident_Index'].count())
print(df1la[df1la.Region=='East']['Accident_Index'].count())
print(df1la[df1la.Region=='North West']['Accident_Index'].count())

In [ ]:
# df_pairplot = df[(df.Date>='2005-01-01')]
# df_pairplot = df_pairplot.groupby([df_pairplot.Date, df_pairplot.Accident_Severity,
#                                   df_pairplot.Hour, df_pairplot.DaysSince,
#                                   df_pairplot.Year])['Accident_Index','Number_of_Casualties','Latitude'].agg(
#                                  {'Accident_Index':'count', 'Number_of_Casualties':'sum',
#                                  'Latitude':'mean'}).reset_index()
# df_pairplot.rename(columns={'Accident_Index': 'Number_of_Daily_Accidents'}, errors='ignore', inplace=True)


# lm = sns.pairplot(df_pairplot[['Number_of_Daily_Accidents','Number_of_Casualties','Hour',
#                           'Year','Latitude','Accident_Severity']], hue='Accident_Severity', 
#                           diag_kind='kde', plot_kws=dict(alpha=1.0))

# axes = lm.axes

# axes[3,0].set_ylim(2004,2018)
# axes[4,3].set_xlim(2004,2018)


# for ax in lm.axes[-1,:]:
#     xlabel = ax.xaxis.get_label_text()
#     ax.xaxis.set_label_text(xlabel,fontsize=12)
# for ax in lm.axes[:,0]:
#     ylabel = ax.yaxis.get_label_text()
#     ax.yaxis.set_label_text(ylabel,fontsize=12)

# plt.setp(lm._legend.get_title(), fontsize=12)

# plt.setp(lm._legend.get_texts(), fontsize=12)

# lm.savefig('pairplot.png')



df1_top15 = df1la[df1la['Region'].isin(df1la['Region'].value_counts()[:15].index)]
df2_top15 = df2la[df2la['Region'].isin(df2la['Region'].value_counts()[:15].index)]
df3_top15 = df3la[df3la['Region'].isin(df3la['Region'].value_counts()[:15].index)]
df4_top15 = df4la[df4la['Region'].isin(df4la['Region'].value_counts()[:15].index)]


fig, ax = plt.subplots(2,2,figsize=(16,12),sharey=True)



sns.countplot(x=df1_top15['Region'],data=df1_top15,
              order = df1_top15['Region'].value_counts().index,ax=ax[0][0])
sns.countplot(x=df2_top15['Region'],
                      data=df2_top15,
                      order = df2_top15['Region'].value_counts().index,
                      ax=ax[0][1])
sns.countplot(x=df3_top15['Region'],
                      data=df3_top15,
                      order = df3_top15['Region'].value_counts().index,
                      ax=ax[1][0])
sns.countplot(x=df4_top15['Region'],
                      data=df4_top15,
                      order = df4_top15['Region'].value_counts().index,
                      ax=ax[1][1])



ax[0][0].yaxis.set_tick_params(labelsize=12)
ax[1][0].yaxis.set_tick_params(labelsize=12)

ax[0][0].set_xticklabels(ax[0][0].get_xticklabels(), rotation=90, fontsize=14)
ax[0][1].set_xticklabels(ax[0][1].get_xticklabels(), rotation=90, fontsize=14)
ax[1][0].set_xticklabels(ax[1][0].get_xticklabels(), rotation=90, fontsize=14)
ax[1][1].set_xticklabels(ax[1][1].get_xticklabels(), rotation=90, fontsize=14)

#ax[0][0].set_yticklabels(ax[0][0].get_yticklabels(), fontsize=14)
#ax[0][1].set_yticklabels(ax[0][1].get_yticklabels(), fontsize=14)
#ax[0][1].set_yticklabels(fontsize=14)
#ax[1][0].set_yticklabels(fontsize=14)
#ax[1][1].set_yticklabels(fontsize=14)



ax[0][0].set_title('Accidents per Region (2006-2008)\n', fontsize=14)
ax[0][1].set_title('Accidents per Region (2009-2011)\n', fontsize=14)
ax[1][0].set_title('Accidents per Region (2012-2014)\n', fontsize=14)
ax[1][1].set_title('Accidents per Region (2015-2017)\n', fontsize=14)

ax[0][0].set_xlabel('')
ax[0][1].set_xlabel('')
ax[1][0].set_xlabel('')
ax[1][1].set_xlabel('')

ax[0][0].set_ylabel('')
ax[0][1].set_ylabel('')
ax[1][0].set_ylabel('')
ax[1][1].set_ylabel('')


fig.tight_layout()
fig.show()


fig.savefig('region.png')

In [ ]:
line_2005 = df_date[(df_date.index=='2005-01-01')]['Accident_Index'].values
line_2006 = df_date[(df_date.index=='2006-01-01')]['Accident_Index'].values
line_2007 = df_date[(df_date.index=='2007-01-01')]['Accident_Index'].values
line_2008 = df_date[(df_date.index=='2008-01-01')]['Accident_Index'].values
line_2009 = df_date[(df_date.index=='2009-01-01')]['Accident_Index'].values
line_2010 = df_date[(df_date.index=='2010-01-01')]['Accident_Index'].values
line_2011 = df_date[(df_date.index=='2011-01-01')]['Accident_Index'].values
line_2012 = df_date[(df_date.index=='2012-01-01')]['Accident_Index'].values
line_2013 = df_date[(df_date.index=='2013-01-01')]['Accident_Index'].values
line_2014 = df_date[(df_date.index=='2014-01-01')]['Accident_Index'].values
line_2015 = df_date[(df_date.index=='2015-01-01')]['Accident_Index'].values
line_2016 = df_date[(df_date.index=='2016-01-01')]['Accident_Index'].values
line_2017 = df_date[(df_date.index=='2017-01-01')]['Accident_Index'].values




In [ ]:
df.head(1)

In [ ]:


# Converting the index as date
df_date.index = pd.to_datetime(df_date.index)

fig, ax = plt.subplots(1,1,figsize=(14,6))

ax = sns.lineplot(x=df_date.index, y='Accident_Index', data=df_date)

years = mdates.YearLocator()   # every year
months = mdates.MonthLocator()  # every month
years_fmt = mdates.DateFormatter('%Y')

# format the ticks
ax.xaxis.set_major_locator(years)
ax.xaxis.set_major_formatter(years_fmt)
ax.xaxis.set_minor_locator(months)

# format the coords message box
ax.format_xdata = mdates.DateFormatter('%Y-%m-%d')
#ax.grid(True)

# round to nearest years.
datemin = np.datetime64(str(df_date.index[0]), 'Y')
datemax = np.datetime64(str(df_date.index[-1]), 'Y') + np.timedelta64(1, 'Y')
ax.set_xlim(datemin, '2011-01-01')

ax.tick_params(which="both", bottom=True,length=6, width=1)
plt.axvline('2006-01-01', color='red', linestyle='dashed')
# plt.axvline(line_2006, color='red', linestyle='dashed')
# plt.axvline(line_2007, color='red', linestyle='dashed')
# plt.axvline(line_2008, color='red', linestyle='dashed')
# plt.axvline(line_2009, color='red', linestyle='dashed')
# plt.axvline(line_2010, color='red', linestyle='dashed')
# plt.axvline(line_2011, color='red', linestyle='dashed')
# plt.axvline(line_2012, color='red', linestyle='dashed')
# plt.axvline(line_2013, color='red', linestyle='dashed')
# plt.axvline(line_2014, color='red', linestyle='dashed')
# plt.axvline(line_2015, color='red', linestyle='dashed')
# plt.axvline(line_2016, color='red', linestyle='dashed')
# plt.axvline(line_2017, color='red', linestyle='dashed')

# rotates and right aligns the x labels, and moves the bottom of the
# axes up to make room for them
fig.autofmt_xdate()
sns.set_style("white")

plt.show()


#fig.savefig('all_accidents.png')

##### Accidents per severity per year

In [ ]:
df_date_sev = df.groupby([df.Date,'Accident_Severity']).count().reset_index()

In [ ]:
df_date_sev

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.dates as mdates



# Converting the index as date
df_date_sev.index = pd.to_datetime(df_date_sev.Date)

fig, ax = plt.subplots(1,1,figsize=(12,4))


# plt.figure(figsize=(14,6))
ax = sns.lineplot(x=df_date_sev[df_date_sev.index<'2007-01-01'].index, y='Accident_Index', hue='Accident_Severity',
             data=df_date_sev[df_date_sev.index<'2007-01-01'],palette=sns.color_palette('bright', df_date_sev.Accident_Severity.unique().shape[0]))



# format the coords message box
ax.format_xdata = mdates.DateFormatter('%Y-%m-%d')
# ax.grid(True)

# round to nearest years.
#datemin = np.datetime64(str(df_date_sev.index[0]), 'Y')
#datemax = np.datetime64(str(df_date_sev.index[-1]), 'Y') + np.timedelta64(1, 'Y')
#ax.set_xlim(datemin, datemax)

years = mdates.YearLocator()   # every year
months = mdates.MonthLocator()  # every month
years_fmt = mdates.DateFormatter('%Y')
# format the ticks
ax.xaxis.set_major_locator(years)
ax.xaxis.set_major_formatter(years_fmt)
ax.xaxis.set_minor_locator(months)
ax.minorticks_on()

ax.tick_params(which="both", bottom=True,length=6, width=1)

#start, end = plt.xlim()
#plt.xticks(np.arange(start, end, (start+end)/10))

# plt.axvline('2005-01-01', color='black', linestyle='dashed')
# plt.axvline('2006-01-01', color='black', linestyle='dashed')
# plt.axvline('2007-01-01', color='black', linestyle='dashed')
# plt.axvline('2008-01-01', color='black', linestyle='dashed')
# plt.axvline('2009-01-01', color='black', linestyle='dashed')
# plt.axvline('2010-01-01', color='black', linestyle='dashed')
# plt.axvline('2011-01-01', color='black', linestyle='dashed')
# plt.axvline('2012-01-01', color='black', linestyle='dashed')
# plt.axvline('2013-01-01', color='black', linestyle='dashed')
# plt.axvline('2014-01-01', color='black', linestyle='dashed')
# plt.axvline('2015-01-01', color='black', linestyle='dashed')
# plt.axvline('2016-01-01', color='black', linestyle='dashed')
# plt.axvline('2017-01-01', color='black', linestyle='dashed')

#fig.autofmt_xdate()
plt.show()

##### Only Serious accidents

In [ ]:
plt.figure(figsize=(16,6))
sns.lineplot(x=df_date_sev[df_date_sev.Accident_Severity=='Serious'].Date, y='Accident_Index',
             data=df_date_sev[df_date_sev.Accident_Severity=='Serious'])
start, end = plt.xlim()
plt.xticks(np.arange(start, end, (start+end)/10))
plt.show()

##### Only fatal accidents

In [ ]:
df_date_sev[df_date_sev.Accident_Severity=='Fatal']

In [ ]:
df11 = df_date_sev
df11.groupby(df11.index.year)['Accident_Index'].sum().reset_index()

In [ ]:
plt.figure(figsize=(16,6))
sns.lineplot(x=df_date_sev[df_date_sev.Accident_Severity=='Fatal'].Date, y='Accident_Index',
             data=df_date_sev[df_date_sev.Accident_Severity=='Fatal'])
start, end = plt.xlim()
plt.xticks(np.arange(start, end, (start+end)/10))
plt.show()

#### Serious/Fatal accidents

In [ ]:
fig, ax = plt.subplots(2,1,figsize=(24,20))

# Converting the index as date
#df_date_sev.index = pd.to_datetime(df_date_sev.Date)
df_date_sev2 = df_date_sev[df_date_sev.Accident_Severity.isin(['Serious','Fatal'])]

ax1 = plt.subplot(211)
sns.lineplot(x=df_date_sev.index, y='Accident_Index', hue='Accident_Severity',
             data=df_date_sev,palette=sns.color_palette('bright', df_date_sev.Accident_Severity.unique().shape[0]),ax=ax1)
# start, end = ax1.set_xlim()
# ax1.set_xticks(np.arange(start, end, (start+end)/10))

years = mdates.YearLocator()   # every year
months = mdates.MonthLocator()  # every month
years_fmt = mdates.DateFormatter('%Y')

# format the ticks
ax1.xaxis.set_major_locator(years)
ax1.xaxis.set_major_formatter(years_fmt)
ax1.xaxis.set_minor_locator(months)

# format the coords message box
ax1.format_xdata = mdates.DateFormatter('%Y-%m-%d')
# ax.grid(True)

# round to nearest years.
datemin = np.datetime64(str(df_date_sev.index[0]), 'Y')
datemax = np.datetime64(str(df_date_sev.index[-1]), 'Y') + np.timedelta64(1, 'Y')
ax1.set_xlim(datemin, datemax)
plt.setp(ax1.get_xticklabels(), fontsize=14)
plt.setp(ax1.get_yticklabels(), fontsize=14)
ax1.set_xlabel('')
ax1.set_ylabel('Accidents per day\n',fontsize=14)
ax1.set_title('\nAll Accidents',fontsize=16)
ax1.tick_params(which="both", bottom=True, length=6, width=1)
plt.legend(fontsize=14)

ax2 = plt.subplot(212)
sns.lineplot(x=df_date_sev2.index, y='Accident_Index',
             data=df_date_sev2,ax=ax2,hue='Accident_Severity')
# start, end = ax2.set_xlim()
# ax2.set_xticks(np.arange(start, end, (start+end)/10))

years = mdates.YearLocator()   # every year
months = mdates.MonthLocator()  # every month
years_fmt = mdates.DateFormatter('%Y')

# format the ticks
ax2.xaxis.set_major_locator(years)
ax2.xaxis.set_major_formatter(years_fmt)
ax2.xaxis.set_minor_locator(months)

# format the coords message box
ax2.format_xdata = mdates.DateFormatter('%Y-%m-%d')
ax2.tick_params(which="both", bottom=True, length=6, width=1)

# round to nearest years.
datemin = np.datetime64(str(df_date_sev2.index[0]), 'Y')
datemax = np.datetime64(str(df_date_sev2.index[-1]), 'Y') + np.timedelta64(1, 'Y')
ax2.set_xlim(datemin, datemax)
plt.setp(ax2.get_xticklabels(), fontsize=14)
plt.setp(ax2.get_yticklabels(), fontsize=14)
ax2.set_title('\nHigh Severity Accidents',fontsize=16)
ax2.set_xlabel('')
ax2.set_ylabel('Accidents per day\n',fontsize=14)


plt.legend(fontsize=14)
plt.show()

fig.savefig('timeline_accidents2.png')

##### Accidents per weekday

### Create weekday category

In [ ]:
cats = [ 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
cat_type = CategoricalDtype(categories=cats, ordered=True)
df['Day_of_Week'] = df['Day_of_Week'].astype(cat_type)

In [ ]:


year_list = [2010,2011,2012,2013,2014,2015,2016,2017]

dfdict = {elem : pd.DataFrame() for elem in year_list}

for i in year_list:
    dfdict[i] = df[df.Year==i].groupby(['Day_of_Week', 'Hour'])['Accident_Index'].count().reset_index().pivot(
                            'Day_of_Week', 'Hour', 'Accident_Index')
        

In [ ]:



fig, axes = plt.subplots(4,2,figsize=(16,10),sharey=True)

ax1, ax2, ax3, ax4, ax5, ax6, ax7, ax8 = axes.ravel()

g1 = sns.heatmap(dfdict[2010], cmap='BuPu',cbar=False,ax=ax1)
g1.set_title("2010 Accidents")

g2 = sns.heatmap(dfdict[2011], cmap='BuPu',cbar=False,ax=ax2)
g2.set_title("2011 Accidents")

g3 = sns.heatmap(dfdict[2012], cmap='BuPu',cbar=False,ax=ax3)
g3.set_title("2012 Accidents")

g4 = sns.heatmap(dfdict[2013], cmap='BuPu',cbar=False,ax=ax4)
g4.set_title("2013 Accidents")

g5 = sns.heatmap(dfdict[2014], cmap='BuPu',cbar=False,ax=ax5)
g5.set_title("2014 Accidents")

g6 = sns.heatmap(dfdict[2015], cmap='BuPu',cbar=False,ax=ax6)
g6.set_title("2015 Accidents")

g7 = sns.heatmap(dfdict[2016], cmap='BuPu',cbar=False,ax=ax7)
g7.set_title("2016 Accidents")

g8 = sns.heatmap(dfdict[2017], cmap='BuPu',cbar=False,ax=ax8)
g8.set_title("2017 Accidents")

fig.tight_layout()
cax,kw = mpl.colorbar.make_axes([ax for ax in axes.flat])
plt.colorbar(axes[0][0].get_children()[0], cax=cax, **kw)

fig.savefig('accidents_heatmap.png')

In [ ]:
cats = [ 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
cat_type = CategoricalDtype(categories=cats, ordered=True)
df['Day_of_Week'] = df['Day_of_Week'].astype(cat_type)

cats = [ 'Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep','Oct', 'Nov', 'Dec']
cat_type = CategoricalDtype(categories=cats, ordered=True)
df['Month'] = df['Month'].astype(cat_type)

year_list = [2015,2016,2017]

dfdictm = {elem : pd.DataFrame() for elem in year_list}

for i in year_list:
    dfdictm[i] = df[df.Year==i & df.Year].groupby(['Day_of_Week', 'Month'])['Accident_Index'].count().reset_index().pivot(
                            'Month', 'Day_of_Week', 'Accident_Index')
        

In [ ]:
acc_events[acc_events.WeekYear==22].groupby('Year')['NAcc'].sum()

In [ ]:
acc_events.Year.value_counts()

In [ ]:
year_hist = [2015,2016,2017]

fig, ax = plt.subplots(1,1,figsize=(25,7))
plt.style.use('ggplot')

#acc_events2 = acc_events.copy()
#acc_events2['Severity'] = np.where(acc_events2.Accident_Severity.isin(['Serious','Fatal']),'Serious/Fatal','Slight')


# r = 1
# for i in year_hist:
#     plt.subplot(5, 1, r) 
#     ax = sns.histplot(data=acc_events[acc_events2.Year == i], x='WeekYear', bins=52, kde=False)
#                  ,hue='Year', hue_order= ['Serious/Fatal','Slight'], palette={'Slight':'b', 'Serious/Fatal':'r'})
#                  #hist_kws=dict(edgecolor="k", linewidth=2), kde=False)   
#     ax.set_xlabel('', fontsize=16)
#     ax.set_ylabel('',fontsize=16)
#     ax.tick_params(labelsize=16)
#     plt.title('Weekly Accidents (' + str(i)+')',fontsize=20)
#     plt.xlim(1, 53)
#     plt.ylim(0, 3800)
#     r+=1

plt.subplot(1, 1, 1) 
ax = sns.histplot(data=acc_events, x='WeekYear', bins=52
             ,hue='Year', hue_order= [2015,2016,2017], palette={2015:'b', 2016:'r', 2017:'g'}
             #,hist_kws=dict(edgecolor="k", linewidth=2, alpha=0.7), 
             ,kde=False)   
ax.set_xlabel('', fontsize=16)
ax.set_ylabel('',fontsize=16)
ax.tick_params(labelsize=16)
plt.title('Weekly Accidents (2015/2016/2017)',fontsize=20)
plt.xlim(1, 53)
plt.ylim(0, 3800)
#r+=1
    
handles, labels = ax.get_legend_handles_labels()
fig.legend(handles, labels, loc='upper left', ncol=1, bbox_to_anchor=(.75, 0.98))

plt.tight_layout()
plt.show()

#fig.savefig('weekly_accidents.png')

In [ ]:

fig, axes = plt.subplots(1,3,figsize=(16,8),sharey=True)

ax1, ax2, ax3 = axes.ravel()

g1 = sns.heatmap(dfdictm[2015], cmap='BuPu',cbar=False,ax=ax1)
g1.set_title("2015 Accidents")
g1.set_xticklabels(g1.get_xticklabels(),fontsize=12)
g1.set_yticklabels(g1.get_yticklabels(), rotation=0,fontsize=12)
g1.set_xlabel('')
g1.set_ylabel('')

g2 = sns.heatmap(dfdictm[2016], cmap='BuPu',cbar=False,ax=ax2)
g2.set_title("2016 Accidents")
g2.set_xticklabels(g2.get_xticklabels(),fontsize=12)
g2.set_xlabel('')
g2.set_ylabel('')

g3 = sns.heatmap(dfdictm[2017], cmap='BuPu',cbar=False,ax=ax3)
g3.set_title("2017 Accidents")
g3.set_xticklabels(g3.get_xticklabels(),fontsize=12)
g3.set_xlabel('')
g3.set_ylabel('')

fig.tight_layout()
cax,kw = mpl.colorbar.make_axes([ax for ax in axes.flat])
plt.colorbar(axes[0].get_children()[0], cax=cax, **kw)

fig.savefig('accidents_heatmap.png')

In [ ]:
year_list = [2015,2016,2017]

dfdictmt = {elem : pd.DataFrame() for elem in year_list}

for i in year_list:
    dfdictmt[i] = df[df.Year==i].groupby(['Day_of_Week','Year'])['Accident_Index'].count().reset_index().pivot(
                            'Day_of_Week', 'Year', 'Accident_Index')
fig, axes = plt.subplots(1,3,figsize=(8,4),sharey=True)

ax1, ax2, ax3 = axes.ravel()

g1 = sns.heatmap(dfdictmt[2015], cmap='BuPu',cbar=False,ax=ax1)
g1.set_title("2015 Accidents")
g1.set_xticklabels(g1.get_xticklabels(),fontsize=12)
g1.set_yticklabels(g1.get_yticklabels(), rotation=0,fontsize=12)
g1.set_xlabel('')
g1.set_ylabel('')

g2 = sns.heatmap(dfdictmt[2016], cmap='BuPu',cbar=False,ax=ax2)
g2.set_title("2016 Accidents")
g2.set_xticklabels(g2.get_xticklabels(),fontsize=12)
g2.set_xlabel('')
g2.set_ylabel('')

g3 = sns.heatmap(dfdictmt[2017], cmap='BuPu',cbar=False,ax=ax3)
g3.set_title("2017 Accidents")
g3.set_xticklabels(g3.get_xticklabels(),fontsize=12)
g3.set_xlabel('')
g3.set_ylabel('')

fig.tight_layout()
cax,kw = mpl.colorbar.make_axes([ax for ax in axes.flat])
plt.colorbar(axes[0].get_children()[0], cax=cax, **kw)

#fig.savefig('accidents_heatmap_overall2.png')

##### Outliers distribution

In [ ]:
df_outlier_y = df.groupby([df.Date,df.Year]).count().reset_index()

In [ ]:
plt.figure(figsize=(16,6))
sns.boxplot(x='Year', y='Accident_Index', data=df_outlier_y)
plt.show()

In [ ]:
sns.boxplot(y='Accident_Index', data=df_outlier_y)

#### Lower end Outliers

In [ ]:
low_outliers = df_outlier_y[df_outlier_y[col] < df_outlier_y[col].mean() 
                        - 3 * df_outlier_y[col].std()]

In [ ]:
low_outliers.sort_values('Accident_Index', ascending = False)[:5].Date.values

##### Days with outliers

In [ ]:
col = 'Accident_Index'
outliers = df_outlier_y[df_outlier_y[col] > df_outlier_y[col].mean() + 3 * df_outlier_y[col].std()]

In [ ]:
col='Accident_Index'
outliers = df_outlier_y[df_outlier_y[col] > df_outlier_y[col].mean() 
                        + 3 * df_outlier_y[col].std()]

In [ ]:
outliers['Accident_Index'].count()

In [ ]:
outliers.nlargest(30, 'Accident_Index').hist('Year')

In [ ]:
out_t = outliers.sort_values('Accident_Index', ascending = False).Date.values
df_out = df[df.Date.isin(out_t)] #.groupby(['Local_Authority_(District)']).count().reset_index()
df_out = df_out[df_out['Local_Authority_(District)'].isin(df_out['Local_Authority_(District)'].value_counts()[:20].index)]

df_outrg = df_out[df_out['Region'].isin(df_out['Region'].value_counts()[:20].index)]

plt.figure(figsize=(14,4))
chart = sns.countplot(x=df_out['Local_Authority_(District)'],data=df_out,
                      order = df_out['Local_Authority_(District)'].value_counts().index)
chart.set_xticklabels(chart.get_xticklabels(), rotation=90)
plt.title('Car Accidents in '+str(out_t)+' (Top 20 Local Authorities)')
plt.show()

##### 1st Outlier

In [ ]:
out1 = outliers.sort_values('Accident_Index', ascending = False)[:1].Date.values
df_1out = df[df.Date.isin(out1)] #.groupby(['Local_Authority_(District)']).count().reset_index()
df_1out = df_1out[df_1out['Local_Authority_(District)'].isin(df_1out['Local_Authority_(District)'].value_counts()[:20].index)]

df_1outrg = df_1out[df_1out['Region'].isin(df_1out['Region'].value_counts()[:20].index)]

In [ ]:
plt.figure(figsize=(14,4))
chart = sns.countplot(x=df_1out['Local_Authority_(District)'],data=df_1out,order = df_1out['Local_Authority_(District)'].value_counts().index)
chart.set_xticklabels(chart.get_xticklabels(), rotation=90)
plt.title('Car Accidents in '+str(out1)+' (Top 20 Local Authorities)')
plt.show()

plt.figure(figsize=(14,4))
chart = sns.countplot(x=df_1outrg['Region'],data=df_1outrg,order = df_1outrg['Region'].value_counts().index)
chart.set_xticklabels(chart.get_xticklabels(), rotation=90)
plt.title('Car Accidents in '+str(out1)+' (Top 20 Regions)')
plt.show()

##### 2nd Outlier

In [ ]:
out2 = outliers.sort_values('Accident_Index', ascending = False)[1:2].Date.values
df_2out = df[df.Date.isin(out2)] #.groupby(['Local_Authority_(District)']).count().reset_index()
df_2out = df_2out[df_2out['Local_Authority_(District)'].isin(df_2out['Local_Authority_(District)'].value_counts()[:20].index)]

df_2outrg = df_2out[df_2out['Region'].isin(df_2out['Region'].value_counts()[:20].index)]

In [ ]:
plt.figure(figsize=(14,4))
chart = sns.countplot(x=df_2out['Local_Authority_(District)'],data=df_2out,order = df_2out
                      ['Local_Authority_(District)'].value_counts().index)
chart.set_xticklabels(chart.get_xticklabels(), rotation=90)
plt.title('Car Accidents in '+str(out2)+' (Top 20 Local Authorities)')
plt.show()

plt.figure(figsize=(14,4))
chart = sns.countplot(x=df_2outrg['Region'],data=df_2outrg,order = df_2outrg['Region'].value_counts().index)
chart.set_xticklabels(chart.get_xticklabels(), rotation=90)
plt.title('Car Accidents in '+str(out2)+' (Top 20 Regions)')
plt.show()

##### 3rd Outlier

In [ ]:
out3 = outliers.sort_values('Accident_Index', ascending = False)[2:3].Date.values
df_3out = df[df.Date.isin(out3)] #.groupby(['Local_Authority_(District)']).count().reset_index()
df_3out = df_3out[df_3out['Local_Authority_(District)'].isin(df_3out['Local_Authority_(District)'].value_counts()[:20].index)]

df_3outrg = df_3out[df_3out['Region'].isin(df_3out['Region'].value_counts()[:20].index)]

In [ ]:
plt.figure(figsize=(14,4))
chart = sns.countplot(x=df_3out['Local_Authority_(District)'],data=df_3out,order = df_3out['Local_Authority_(District)'].value_counts().index)
chart.set_xticklabels(chart.get_xticklabels(), rotation=90)
plt.title('Car Accidents in '+str(out3)+' (Top 20 Local Authorities)')
plt.show()

plt.figure(figsize=(14,4))
chart = sns.countplot(x=df_3outrg['Region'],data=df_3outrg,order = df_3outrg['Region'].value_counts().index)
chart.set_xticklabels(chart.get_xticklabels(), rotation=90)
plt.title('Car Accidents in '+str(out3)+' (Top 20 Regions)')
plt.show()

#### Outliers in Birmingham

In [ ]:
birg_out

In [ ]:
birg_out = outliers.sort_values('Accident_Index', ascending = False)[:3].Date.values
df[(df.Date.isin(birg_out)) & (df['Local_Authority_(District)']=='Birmingham')].groupby(
    ['Accident_Severity','Day_of_Week'])['Accident_Index'].count()


##### Analysis of Birmingham accidents (Serious/Fatal)

In [ ]:
out10s = outliers.sort_values('Accident_Index', ascending = False)[:10].Date.values
df_10out = df[df.Date.isin(out10s) & (df['Local_Authority_(District)']=='Birmingham') &
              (df.Accident_Severity!='Slight')]

In [ ]:
df_10out['Accident_Index'].count()

In [ ]:
plt.figure(figsize=(14,4))
ax = sns.barplot(x="Date", y="Number_of_Casualties", hue="Accident_Severity", data=df_10out)
#plt.plot(x='Date',y='Number_of_Casualties',data=df_10out)

In [ ]:
plt.figure(figsize=(14,4))
ax = sns.barplot(x="Date", y="Number_of_Casualties", hue="Hour", data=df_10out)

### Ad-hoc

In [ ]:
b = df[((df.Date>='2015-01-01') & (df.Date<='2017-12-31'))
                  & (df.Region=='London')
                  & (df.Number_of_Casualties>1)]

In [ ]:
plt.figure(figsize=(14,4))
chart = sns.countplot(x=b['Local_Authority_(District)'],
                      data=b,order = b['Local_Authority_(District)'].value_counts().index)
chart.set_xticklabels(chart.get_xticklabels(), rotation=90)
#plt.title('Car Accidents in '+str(out3)+' (Top 20 Local Authorities)')
plt.show()


### Histogram of dates in each year (2013-2017)

In [ ]:
acc_events['NAcc'] = 1
acc_events[(acc_events.Month=='Jan') & (acc_events.Year==2015)].groupby('Date')['NAcc'].sum()

In [ ]:
#df_analysis = df[df.Year >= 2013]
acc_events = df[df.Year >= 2013]

In [ ]:
#year_hist = [2013,2014,2015,2016,2017]

year_hist = [2015,2016,2017]

plt.figure(figsize=(20,20))
plt.style.use('ggplot')

acc_events2 = acc_events.copy()
acc_events2['Severity'] = np.where(acc_events2.Accident_Severity.isin(['Serious','Fatal']),'Serious/Fatal','Slight')


r = 1
for i in year_hist:
    plt.subplot(5, 1, r) 
    ax = sns.histplot(data=acc_events2[acc_events2.Year == i], x='WeekYear', bins=52, kde=False)
                 #,hue='Severity', hue_order= ['Serious/Fatal','Slight'], palette={'Slight':'b', 'Serious/Fatal':'r'})
                 #hist_kws=dict(edgecolor="k", linewidth=2), kde=False)   
    plt.title('Weekly Accidents in ' + str(i))
    plt.xlim(1, 53)
    plt.ylim(0, 3800)
    r+=1

handles, labels = ax.get_legend_handles_labels()
fig.legend(handles, labels, loc='upper left', ncol=1, bbox_to_anchor=(.75, 0.98))

plt.tight_layout()
plt.show()

### Times of the day x Number of casualties (year by year)

In [ ]:
acc_events.head(1)

In [ ]:
#year_hist = [2013,2014,2015,2016,2017]

plt.figure(figsize=(16,9))
plt.style.use('ggplot')

# ax = sns.scatterplot(data=acc_events,
#                 x='Hour',y='Number_of_Casualties',hue='Year',alpha=0.7,
#                 palette="colorblind")

ax = sns.relplot(data=acc_events_2,
                x='Hour',y='Number_of_Casualties',hue='Year',alpha=0.7,
                palette="colorblind",kind='scatter',col='Severity'
)

FacetGrid.set(xticks=np.arange(0,24,1))

#ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
ax.set_xlim(-0.5,24)

plt.show()

### London

In [ ]:
#df_temp = df[(df.Region=='London') & (df.Year>=2006)]
#df_temp = df[(df.Accident_Severity!='Slight') & (df.Year>=2005)]
df_temp = df[(df.Accident_Severity!='Slight') & (df.Year>=2005)]

In [ ]:
df['Lightness'] = np.where(df.Light_Conditions=='Daylight','Daylight',
                           np.where(df.Light_Conditions.str.contains('Darkness'),'Darkness',''))

In [ ]:
df.Month.value_counts()

In [ ]:
df_temp_h = df_temp.groupby(['Year','Month','Day_of_Week','Hour']).count().reset_index()
df_temp_h = df_temp_h.groupby('Hour').mean().reset_index()
df_temp_h.head(1)

In [ ]:
df_temp_y = df_temp.groupby(['Year']).count().reset_index()

# df_temp_m = df_temp.groupby(['Year','Month','Lightness']).count().reset_index()
# df_temp_m = df_temp_m.groupby(['Month','Lightness']).mean().reset_index()

df_temp_m = df_temp.groupby(['Year','Month']).count().reset_index()
df_temp_m = df_temp_m.groupby(['Month']).mean().reset_index()

df_temp_w = df_temp.groupby(['Year','Month','Day_of_Week']).count().reset_index()
df_temp_w = df_temp_w.groupby('Day_of_Week').mean().reset_index()

df_temp_h = df_temp.groupby(['Year','Month','Day_of_Week','Hour']).count().reset_index()
df_temp_h = df_temp_h.groupby('Hour').mean().reset_index()

In [ ]:

f, axes = plt.subplots(2, 2, figsize=(18,9))

Months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 
             'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']

Weekdays = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 
           'Friday', 'Saturday', 'Sunday']

sns.barplot(x='Year',y='Accident_Index',data=df_temp_y,ax=axes[0][0])
sns.barplot(x='Month',y='Accident_Index',data=df_temp_m,order=Months,ax=axes[0][1])
#sns.barplot(x='Month',y='Accident_Index',data=df_temp_m,order=Months,ax=axes[0][1],hue='Lightness')
sns.barplot(x='Day_of_Week',y='Accident_Index',data=df_temp_w,order=Weekdays,ax=axes[1][0])
sns.barplot(x='Hour',y='Accident_Index',data=df_temp_h,ax=axes[1][1])

axes[0][0].set_title('Number of Accidents per Year')
axes[0][1].set_title('Avg of Accidents by Month (2005-2017)')
axes[1][0].set_title('Avg of Accidents by Day of Week (2005-2017)')
axes[1][1].set_title('Avg of Accidents by Hour (2005-2017)')

axes[0][0].set_ylabel('')
axes[0][1].set_ylabel('')
axes[1][0].set_ylabel('')
axes[1][1].set_ylabel('')

axes[1][0].set_xlabel('Year')
axes[1][0].set_xlabel('Day of Week')
axes[1][0].set_xlabel('Day of Week')
axes[1][0].set_xlabel('Day of Week')

plt.tight_layout()
plt.show()

#f.savefig('fig12c.png')

## Spatial Analysis

In [ ]:
acc_events_201301['Latitude']

In [ ]:
ctr_pol_en = acc_events_201301[acc_events_201301.Country=='England']['Latitude']
ctr_pol_wa = acc_events_201301[acc_events_201301.Country=='Wales']['Latitude']
ctr_pol_sc = acc_events_201301[acc_events_201301.Country=='Scotland']['Latitude']


hr_pol_en = acc_events_201301[acc_events_201301.Country=='England']['Hour']
hr_pol_wa = acc_events_201301[acc_events_201301.Country=='Wales']['Hour']
hr_pol_sc = acc_events_201301[acc_events_201301.Country=='Scotland']['Hour']

In [ ]:

# set up random data between 0 and 90
#r = [np.random.random() * 90.0 for i in range(0,10)]
#r = .tolist()

# set up 24 hours matching the random data above
#hours = np.linspace(0.0,24.0,len(r))
hours_en = np.array(hr_pol_en)
hours_wa = np.array(hr_pol_wa)
hours_sc = np.array(hr_pol_sc)

# scaling the 24 hours to the full circle, 2pi
theta_en = hours_en / 24.0 * (2.0 * np.pi)
theta_wa = hours_wa / 24.0 * (2.0 * np.pi)
theta_sc = hours_sc / 24.0 * (2.0 * np.pi)


# reverse your data, so that 90 becomes 0:
#r_rev = [(ri - 90.0) * -1.0 for ri in r]
r_rev_en = ctr_pol_en.tolist()
r_rev_wa = ctr_pol_wa.tolist()
r_rev_sc = ctr_pol_sc.tolist()

plt.figure(figsize=(22, 12))
# set up your polar plot
ax = plt.subplot(111, projection='polar')
ax.plot(theta_en, r_rev_en, color='r', linewidth=0.40, alpha=0.6)
ax.plot(theta_wa, r_rev_wa, color='k', linewidth=0.50, alpha=0.8)
ax.plot(theta_sc, r_rev_sc, color='b', linewidth=0.40, alpha=0.7)

# define your axis limits
ax.set_ylim([40.0, 61.0])

# statically reverse your y-tick-labels
# caution: this turns your labels into strings
#          and decouples them from the data
# 
# the np.linspace gives you a distribution between 90 and 0 -
# the number of increments are related to the number of ticks
# however, you require one more label, because the center is 
#     omitted.  
ax.set_yticklabels(['{:.0f}'.format(ylabel) \
                for ylabel in np.linspace(40.0,61.0,len(ax.get_yticklabels())+1)[1:]])


# statically turn your x-tick-labels into fractions of 24
# caution: this turns your labels into strings
#          and decouples them from the data
#
# the number of ticks around the polar plot is used to derive
#    the appropriate increment for the 24 hours
ax.set_xticklabels(['{:.1f}'.format(xlabel) \
                    for xlabel in np.arange(0.0,24.0,(24.0 / len(ax.get_xticklabels())))])

ax.grid(True)

plt.show()

In [ ]:
g = sns.FacetGrid(df_dt, #the dataframe to pull from
                  row="Year", #define the column for each subplot row to be differentiated by
                  hue="Year", #define the column for each subplot color to be differentiated by
                  aspect=5, #aspect * height = width
                  height=1.5, #height of each subplot
                  palette=['#4285F4','#EA4335','#FBBC05','#34A853'] #google colors
                 )

In [ ]:
g.map(sns.kdeplot, "Accident_Index", shade=True, alpha=1, lw=1.5, bw_method=0.2)
g.map(sns.kdeplot, "Accident_Index", lw=4, bw=0.2)

##### Spatial distribution of May/2020 accidents considering daily events with maximum duration

In [ ]:
df_events_jun20 = df.groupby('Date')[['State','Start_Lat','Start_Lng','Duration_Minutes']].agg(
    {'Duration_Minutes':['max'], 'State':['first'],
    'Start_Lat':['first'],'Start_Lng':['first']}).reset_index(level=0, drop=True)

In [ ]:
df_events_jun20 = df_events_jun20.loc['2020-01-01':'2020-02-01']

In [ ]:
df_events_jun20.info()

In [ ]:
m = folium.Map(location=[45.5236, -122.6750])

In [ ]:
# add markers to map
for lat, lng, state in zip(df_events_jun20['Start_Lat'], df_events_jun20['Start_Lng'], df_events_jun20['State']):
    label = 'Name: {0}'.format(state)
    iframe = folium.IFrame(html=label, width=300, height=100)
    popup = folium.Popup(iframe, parse_html=True)
    folium.CircleMarker([lat, lng],
                        radius=5,
                        popup=popup,
                        color='blue',
                        fill=True,
                        fill_color='blue',
                        fill_opacity=0.7,
                       ).add_to(m)

m

In [ ]:
#step to linear using .to_linear()
colorscale = branca.colormap.step.RdYlBu_11.to_linear().scale(0, 30)
colorscale.caption = 'Color Scale'
def style_function(feature):
    employed = employed_series.get(int(feature['id'][-5:]), None)
    return {
        'fillOpacity': 0.5,
        'weight': 0,
        'fillColor': '#black' if employed is None else colorscale(employed)
    }
us_county_lin = folium.Map(
    location=[42, -100],
    tiles='cartodbpositron',
    zoom_start=4
)
folium.TopoJson(
    json.loads(requests.get(counties).text),
    'objects.us_counties_20m',
    style_function=style_function
).add_to(us_county_lin)
colorscale.add_to(us_county_lin)
us_county_lin

## Spatial distribution of accidents events

In [ ]:
#basedate = pd.to_datetime(acc_events.Date.min())

In [ ]:
#acc_events['days_since'] = (acc_events['Datetime'] - pd.to_datetime(acc_events.Date.min())).dt.days

In [ ]:
# df[((df.Date>='2015-01-01') & (df.Date<='2017-12-31'))
#                   & (df.Region=='London')
#                   & (df.Number_of_Casualties>1)].groupby(['Local_Authority_(District)']).count()

In [ ]:

m = folium.Map(location=[54.584797 , -3.438721], zoom_start=6,
               tiles='cartodbpositron', width='100%', height='100%') 
# If you adjusted the notebook display width to be as wide as your screen, the map might get very big. 
# Adjust size as desired.



for id, row in df[((df.Date>='2015-10-01') & (df.Date<='2017-12-31'))
                  & (df.Number_of_Casualties>1)].iterrows():
    #if row['Accident_Severity']!='Slight':
#         label = 'Name: {0}'.format(row['Datetime'])
#         iframe = folium.IFrame(html=label, width=300, height=100)
#         popup = folium.Popup(iframe, parse_html=True)
        folium.CircleMarker((row.Latitude, row.Longitude),
                            radius=row.Number_of_Casualties*0.2, color='b', 
                            fill=True, fill_opacity=0.6, #opacity=0,
                            fill_color='red').add_to(m)
    #else:
    #    folium.CircleMarker((row.Latitude, row.Longitude),
    #                    radius=row.Number_of_Casualties*0.2, color='b', 
    #                        fill=True, fill_opacity=0.6, #opacity=0,  
    #                   fill_color='blue').add_to(m)
    
    
    
# for id, row in df[((df.Date>='2015-01-01') & (df.Date<='2015-03-31'))
#               & (df.Accident_Severity=='Slight')
#               & (df.Number_of_Casualties>1)].iterrows():
#     folium.CircleMarker((row.Latitude, row.Longitude),
#                     radius=2, color='b', fill=True, fill_opacity=0.5, #opacity=0,  
#                     fill_color='blue').add_to(m)

 


m

#m.save(os.path.join('', 'f_3yn_accidents.html'))

In [ ]:

delay=5
 
#Save the map as an HTML file
fn='testmap.html'
tmpurl='file://{path}/{mapfile}'.format(path=os.getcwd(),mapfile=fn)
m.save(fn)

#Open a browser window...
browser = webdriver.Chrome()
#..that displays the map...
browser.get(tmpurl)
#Give the map tiles some time to load
time.sleep(delay)
#Grab the screenshot
browser.save_screenshot('map2.png')
#Close the browser
browser.quit()

Large clusters can be seen around Greater London, Birmingham, Manchester, Leeds, Newcastle upton Tyne and Edinburgh and Glasgow

### London

In [ ]:
out_lon = dflon.sort_values(by=['Accident_Index'],ascending=False).nlargest(10,'Accident_Index')['Date'].values

In [ ]:
dflon = df[((df.Date>='2010-01-01')) #& (df.Date<='2017-12-31'))
                  & (df.Region=='London')].groupby('Date')['Accident_Index'].count().reset_index()


dflon.plot('Date','Accident_Index')

In [ ]:
out_lon

In [ ]:
m = folium.Map(location=[51.484797 , -0.138721],
               zoom_start=10,
               tiles='cartodbpositron', width='100%', height='100%') 
# If you adjusted the notebook display width to be as wide as your screen, the map might get very big. 
# Adjust size as desired.

for id, row in df[((df.Date=='2016-11-25')) #& (df.Date<='2017-12-31'))
                  & (df.Region=='London')].iterrows():
    #if row['Accident_Severity']!='Slight':
        label = 'Date: {0}'.format(row['Datetime'])
        iframe = folium.IFrame(html=label, width=300, height=100)
        popup = folium.Popup(iframe, parse_html=True)
        folium.CircleMarker((row.Latitude, row.Longitude),
                            radius=3, color='b', fill=True, fill_opacity=1, popup=popup, #opacity=0,
                            fill_color='red').add_to(m)
    #else:
    #    folium.CircleMarker((row.Latitude, row.Longitude),
    #                    radius=3, color='b', fill=True, fill_opacity=1, #opacity=0,  
    #                   fill_color='blue').add_to(m)

    
# sw = df[['Latitude', 'Longitude']].min().values.tolist()
# ne = df[['Latitude', 'Longitude']].max().values.tolist()

# m.fit_bounds([sw, ne]) 
    
m

#m.save(os.path.join('', 'f_trim_2013_lon_accidents.html'))

In [ ]:
#Save the map as an HTML file
fn='testmap.html'
tmpurl='file://{path}/{mapfile}'.format(path=os.getcwd(),mapfile=fn)
m.save(fn)

#Open a browser window...
browser = webdriver.Chrome()
#..that displays the map...
browser.get(tmpurl)
#Give the map tiles some time to load
time.sleep(delay)
#Grab the screenshot
browser.save_screenshot('mapldn.png')
#Close the browser
browser.quit()

In [ ]:
df[((df.Date>='2005-01-01')) & (df.Date<='2017-12-31')
                  & (df.Region=='London')
                  & (df.Accident_Severity=='Fatal')
                  & (df.Number_of_Casualties>1)].count()

In [ ]:
from folium.features import DivIcon


m = folium.Map(location=[51.504797 , -0.068721],
               zoom_start=12,
               tiles='cartodbpositron', width='100%', height='100%') 
# If you adjusted the notebook display width to be as wide as your screen, the map might get very big. 
# Adjust size as desired.


df_lon_mul = df[((df.Date>='2015-01-01') & (df.Date<='2017-12-31'))
                  & (df.Region=='London')
                  & (df.Number_of_Casualties>1)]
df_lon_mul=df_lon_mul.groupby(['Hour'])['Accident_Index'].agg(
    # make the numbers numeric otherwise it just concatenates strings
    lambda x: pd.to_numeric(x, errors='coerce').count()
)

hour_dict = df_lon_mul.to_dict()

colormap = branca.colormap.linear.YlOrRd_09.scale(0, 24)
colormap = colormap.to_step(index=[0, 8, 14, 20, 24])
colormap.caption = 'Hours of Accidents in London (2015-2017)'
#'2015-10-01'
#text = 'Test'
for id, row in df[((df.Date>='2005-01-01')) & (df.Date<='2017-12-31')
                  & (df.Region=='London')
                  & (df.Accident_Severity=='Fatal')
                  & (df.Number_of_Casualties>1)
                 ].iterrows():
        folium.CircleMarker((row.Latitude, row.Longitude),
                            radius=row.Number_of_Casualties, 
                            popup='Date of the accident:'+str(row.Datetime),
                            fill=True,
                            color='b',
                            fill_color = colormap(colormap.index[1] if ((row.Hour>=0) & (row.Hour<8)) 
                                                  else colormap.index[2]
                                                  if (row.Hour>=8) & (row.Hour<14) 
                                                  else colormap.index[3]
                                                  if (row.Hour>=14) & (row.Hour<20) 
                                                  else colormap.index[4])
                            ,fill_opacity=1).add_to(m)
        
#         folium.map.Marker(
#                         (row.Latitude, row.Longitude),
#                         icon=DivIcon(
#                             icon_size=(150,36),
#                             icon_anchor=(0,0),
#                             html='<div style="font-size: 12pt">%s</div>' % str('Date of the accident:'+str(row.Datetime)),
#                             )
#                         ).add_to(m)
        
m.add_child(colormap)
        
m

In [ ]:
from folium.features import DivIcon


m = folium.Map(location=[51.504797 , -0.068721],
               zoom_start=12,
               tiles='cartodbpositron', width='100%', height='100%') 
# If you adjusted the notebook display width to be as wide as your screen, the map might get very big. 
# Adjust size as desired.


df_lon_mul = df[((df.Date>='2015-01-01') & (df.Date<='2017-12-31'))
                  & (df.Region=='London')
                  & (df.Number_of_Casualties>1)]
df_lon_mul=df_lon_mul.groupby(['Hour'])['Accident_Index'].agg(
    # make the numbers numeric otherwise it just concatenates strings
    lambda x: pd.to_numeric(x, errors='coerce').count()
)

hour_dict = df_lon_mul.to_dict()

colormap = branca.colormap.linear.YlOrRd_09.scale(0, 24)
colormap = colormap.to_step(index=[0, 8, 14, 20, 24])
colormap.caption = 'Hours of Accidents in London (2015-2017)'
#'2015-10-01'
#text = 'Test'
for id, row in df[((df.Date>='2017-01-01')) & (df.Date<='2017-12-31')
                  & (df.Region=='London')
                  #& (df.Accident_Severity=='Serious')
                 ].iterrows():
        folium.CircleMarker((row.Latitude, row.Longitude),
                            radius=row.Number_of_Casualties, 
                            popup='Date of the accident:'+str(row.Datetime),
                            fill=True,
                            color='b',
                            fill_color='b'
#                             fill_color = colormap(colormap.index[1] if ((row.Hour>=0) & (row.Hour<8)) 
#                                                   else colormap.index[2]
#                                                   if (row.Hour>=8) & (row.Hour<14) 
#                                                   else colormap.index[3]
#                                                   if (row.Hour>=14) & (row.Hour<20) 
#                                                   else colormap.index[4])
                            ,fill_opacity=1).add_to(m)
        
#         folium.map.Marker(
#                         (row.Latitude, row.Longitude),
#                         icon=DivIcon(
#                             icon_size=(150,36),
#                             icon_anchor=(0,0),
#                             html='<div style="font-size: 12pt">%s</div>' % str('Date of the accident:'+str(row.Datetime)),
#                             )
#                         ).add_to(m)
        
m.add_child(colormap)
        
m

In [ ]:
#Save the map as an HTML file
fn='testmap.html'
tmpurl='file://{path}/{mapfile}'.format(path=os.getcwd(),mapfile=fn)
m.save(fn)

#Open a browser window...
browser = webdriver.Chrome()
#..that displays the map...
browser.get(tmpurl)
#Give the map tiles some time to load
time.sleep(delay)
#Grab the screenshot
browser.save_screenshot('mapldn_hr2.png')
#Close the browser
browser.quit()

In [ ]:
df_lon_mul = df[((df.Date>='2015-01-01') & (df.Date<='2017-12-31'))
                  & (df.Region=='London')
                  & (df.Number_of_Casualties>1)]

fig = plt.figure(figsize=(16, 6))

n, bins, patches = plt.hist(df_lon_mul['Hour'], 23)

for c, p in zip(bins, patches):
    if round(c,0) >= 0 and round(c,0) < 8:
        plt.setp(p, 'facecolor', '#ffffccff')
    elif round(c,0) >= 8 and round(c,0) < 14:
        plt.setp(p, 'facecolor', '#fea646ff')
    elif round(c,0) >= 14 and round(c,0) < 20:
        plt.setp(p, 'facecolor', '#e31a1cff')
    else:
        plt.setp(p, 'facecolor', '#800026ff')

plt.xticks(np.arange(0, 24, step=1))
plt.xlim(0,23)
plt.xlabel('Hour')
plt.ylabel('count')
plt.show()
fig.savefig('histldn_hr.png')

### Speed limit map

In [ ]:


m = folium.Map(location=[51.504797 , -0.068721],
               zoom_start=11,
               tiles='cartodbpositron', width='100%', height='100%') 
# If you adjusted the notebook display width to be as wide as your screen, the map might get very big. 
# Adjust size as desired.

colormap = cm.LinearColormap(colors=['#FFD0C2','#FF8A83','#D65F59','#C23210','#991101','#680101'], 
                             index=[20, 30, 40, 50, 60, 70],vmin=20,vmax=70)
colormap = colormap.to_step(index=[20, 30, 40, 50, 60, 70])
colormap.caption = 'Road Speed limit of Accidents in London (2015-2017)'

for id, row in df[((df.Date>='2015-01-01') & (df.Date<='2017-12-31'))
                  & (df.Region=='London')
                  & (df.Number_of_Casualties>1)].iterrows():
        folium.CircleMarker((row.Latitude, row.Longitude),
                            radius=row.Number_of_Casualties, color='b',
                            popup='Speed limit of the road:'+str(row.Speed_limit),
                            fill_color = '#FFD0C2' if row.Speed_limit==20 else '#FF8A83' if row.Speed_limit==30 else '#D65F59' 
                            if row.Speed_limit==40 else '#C23210' if row.Speed_limit==50 else
                            '#991101' if row.Speed_limit==60 else '#680101'
                            ,fill_opacity=0.4).add_to(m)

m.add_child(colormap)
        
#m

In [ ]:
#Save the map as an HTML file
fn='testmap.html'
tmpurl='file://{path}/{mapfile}'.format(path=os.getcwd(),mapfile=fn)
m.save(fn)

#Open a browser window...
browser = webdriver.Chrome()
#..that displays the map...
browser.get(tmpurl)
#Give the map tiles some time to load
time.sleep(delay)
#Grab the screenshot
browser.save_screenshot('mapldn_spd.png')
#Close the browser
browser.quit()

In [ ]:
df_lon_mul = df[((df.Date>='2015-01-01') & (df.Date<='2017-12-31'))
                  & (df.Region=='London')
                  & (df.Number_of_Casualties>1)]

fig, ax = plt.subplots(1,1,figsize=(16, 6))

ax = sns.countplot(x='Speed_limit', data=df_lon_mul,
                   palette=['#FFD0C2','#FF8A83','#D65F59',
                            '#C23210','#991101','#680101'])

# for c, p in zip(bins, patches):
#     print(c,p)
#     if c==20:
#         plt.setp(p, 'facecolor', '#FFD0C2')
#     elif c==30:
#         plt.setp(p, 'facecolor', '#FF8A83')
#     elif c==40:
#         plt.setp(p, 'facecolor', '#D65F59')
#     if c==50:
#         plt.setp(p, 'facecolor', '#C23210')
#     elif c==60:
#         plt.setp(p, 'facecolor', '#991101')
#     else:
#         plt.setp(p, 'facecolor', '#680101')


plt.show()
fig.savefig('histldn_spd.png')

### Putting the data on a map

In [ ]:
acc_events_201301 = acc_events[((acc_events.Date>='2013-01-01') &
                               (acc_events.Date<'2013-02-01')) & (acc_events.Number_of_Casualties > 1)]

In [ ]:
acc_events_201301.Number_of_Casualties.mean()

In [ ]:
acc_events_2014.head(1)

In [ ]:
# c is the attribute we'll map onto colors, s is the attribute we'll represent with circle size.
acc_events_201301.plot(kind='scatter', x='Longitude', y='Latitude',
    s=acc_events_201301['Number_of_Casualties']*20, label='Number_of_Casualties',
    c='Hour', cmap=plt.get_cmap("jet"),
    colorbar=True, alpha=0.5, figsize=(10,10),
)
plt.legend()
#save_fig("housing_prices_scatterplot")
plt.show()

### Overview of the spatial distribution in 3D (2D space and 1D Time)

In [ ]:

fig = plt.figure(1, figsize=(9, 6))

acc_events_2014 = acc_events[(acc_events.Year==2014)]

ax = Axes3D(fig, rect=[0, 0, .95, 5], elev=25, azim=115) # change parameters here for looking from a different perspective
ax.scatter(acc_events_2014['Longitude'], acc_events_2014['Latitude'], 
           acc_events_2014['days_since'] #,c=acc_events_2014['days_since'], cmap='Greens'
           , alpha=0.1, c=acc_events_2014['days_since'], cmap='viridis')
plt.show()

## Default configuration for DBSCAN

The scikit-learn DBSCAN haversine distance metric requires data in the form of [latitude, longitude] and both inputs and outputs are in units of radians.

In [ ]:


# define the number of kilometers in one radian
kms_per_radian = 6371.0088
#spatial_dist_max = 5 / kms_per_radian
#temporal_dist_max = 24

### Compute DBSCAN

* eps is the physical distance from each point that forms its neighborhood
* min_samples is the min cluster size, otherwise it's noise - set to 1 so we get no noise

In [ ]:


# represent points consistently as (lat, lon)
coords = acc_events_201301[['Longitude','Latitude']].values

In [ ]:
##### spatial_dist_max = 25 / kms_per_radian
temporal_dist_max = 24
n_neighbours = 3

# define epsilon as 1.5 kilometers, converted to radians for use by haversine
eps = 1.5 / kms_per_radian

clustered_ST = DBSCAN(eps=eps,metric='haversine', 
                      min_samples=n_neighbours).fit(np.radians(coords))

print("Clustering finished!")

labels=clustered_ST.labels_
unique_labels=np.unique(clustered_ST.labels_)
print('Result: {} records in the noise, labelled as -1, and {} clusters labelled as 0..{}'.
      format(acc_events_201301[labels==-1].shape[0], len(unique_labels)-1, len(unique_labels)-2))
#clustered
clust_id_col_name='ClusterN'
acc_events_201301[clust_id_col_name]=labels


In [ ]:
# Getting cluster sizes
cluster_sizes = acc_events_201301[clust_id_col_name].value_counts().rename_axis('Cluster id').to_frame('count')
print("Cluster sizes:")
print(cluster_sizes.head(15))
print("...")
print(cluster_sizes.tail(15))

cluster_sizes = cluster_sizes[cluster_sizes.index != -1] # no noise

max_cluster_size=cluster_sizes['count'].max()
print("max = ",max_cluster_size)

In [ ]:
agg_func = {
    'days_since':['max','min'],
    'Longitude':['mean','max','min'],
    'Latitude':['mean','max','min']
}
st_aggregates = acc_events_201301.reset_index(drop=False)[['ClusterN','days_since',
                                                'Longitude','Latitude']].groupby(['ClusterN']).agg(agg_func)
# Flatten hierarchical column names
st_aggregates.columns = ["_".join(x) for x in st_aggregates.columns.ravel()]
# compute derived attributes: duration and bounding rectangle diagonal
st_aggregates['duration (days)']=st_aggregates['days_since_max']-st_aggregates['days_since_min']
for id,row in st_aggregates.iterrows():
    brd=kms_per_radian*great_circle2(row['Latitude_max'],row['Longitude_max'],row['Latitude_min'],row['Longitude_min'])
    #print('{}'.format(brd))
    #print(row['Latitude_max'],row['Longitude_max'],row['Latitude_min'],row['Longitude_min'])
    st_aggregates.at[id,'Bound_rect_diag(km)']=brd
st_aggregates

In [ ]:


clusters_data = st_aggregates.loc[st_aggregates.index!=-1,
                                  ['Latitude_mean','Longitude_mean',
                                   'days_since_min','days_since_max']]

scaler = MinMaxScaler()
clusters_data_scaled = scaler.fit_transform(clusters_data)

mds_ST = MDS(n_components = 2, random_state=110)
mds_ST.fit(clusters_data_scaled)
xy_mds_ST = mds_ST.fit_transform(clusters_data_scaled)

xmin_ST=xy_mds_ST[:,0].min() 
xmax_ST=xy_mds_ST[:,0].max()
ymin_ST=xy_mds_ST[:,1].min()
ymax_ST=xy_mds_ST[:,1].max()
print(xmin_ST,xmax_ST,ymin_ST,ymax_ST)

### Plot spatial distribution on the map

In [ ]:


lon_range = (-130.60, -52.75)
lat_range = (17.13, 53.65)
m = folium.Map(tiles='cartodbpositron', width='100%', height='100%') 
# If you adjusted the notebook display width to be as wide as your screen, the map might get very big. 
#Adjust size as desired.
m.fit_bounds([[lat_range[0], lon_range[0]], [lat_range[1], lon_range[1]]])
for id, row in acc_events_201301.iterrows():
    cluster_id = row[clust_id_col_name]
    if cluster_id != -1 and len(np.where(clusters_data.index==cluster_id)[0])>0:
        i=np.where(clusters_data.index==cluster_id)[0][0]
        if i<len(xy_mds_ST):
            folium.CircleMarker((row['Latitude'], row['Longitude']), radius=2, 
                        #color=clust_colors[cluster_id % len(clust_colors)], 
                        color=getColor(xy_mds_ST[i,0], xy_mds_ST[i,1],xmin_ST,xmax_ST,ymin_ST,ymax_ST),
                        fill=False, opacity=.3,
                        popup='Cluster: {}'.format(cluster_id)).add_to(m)
            
m

## 4.1 Clustering of UK accidents

In [ ]:
acc_evt_hsall = df[
             #(df['Local_Authority_(District)'].isin(london_borough)) &
             (df.Accident_Severity !='Slight') &
             #(df.Number_of_Casualties > 1) &
             (df.Year >= 2017)]
acc_evt_hsall.shape

In [ ]:
X_hsall  = acc_evt_hsall[['Longitude','Latitude']].values

### Knee Locator

In [ ]:

nearest_neighbors = NearestNeighbors(n_neighbors=7)
neighbors = nearest_neighbors.fit(X_hsall)
distances, indices = neighbors.kneighbors(X_hsall)
distances = np.sort(distances[:,6], axis=0)



i = np.arange(len(distances))
knee = KneeLocator(i, distances, S=1, curve='convex', direction='increasing', interp_method='polynomial')
fig = plt.figure(figsize=(5, 5))
knee.plot_knee()
plt.xlabel("Points")
plt.ylabel("Distance")

print(distances[knee.knee])
#plt.savefig("Distance_curve.png", dpi=300)

In [ ]:
n_neighbours = 7

# represent points consistently as (lat, lon)
coords = acc_evt_hsall[['Longitude','Latitude']]


# define epsilon as 1.5 kilometers, converted to radians for use by haversine
spatial_dist_max = distances[knee.knee] #/ kms_per_radian
#spatial_dist_max = 0.00006

clustered_ST = DBSCAN(eps=np.radians(distances[knee.knee]),
                      metric='haversine',min_samples=n_neighbours).fit(np.radians(coords))

print("Clustering finished!")

labels=clustered_ST.labels_
unique_labels=np.unique(clustered_ST.labels_)
print('Result: {} records in the noise, labelled as -1, and {} clusters labelled as 0..{}'.
      format(acc_evt_hsall[labels==-1].shape[0], len(unique_labels)-1, len(unique_labels)-2))

#clustered
clust_id_col_name='ClusterN'
acc_evt_hsall[clust_id_col_name]=labels

In [ ]:
## Getting cluster sizes
cluster_sizes = acc_evt_hsall[clust_id_col_name].value_counts().rename_axis('Cluster id').to_frame('count')
print("Cluster sizes:")
print(cluster_sizes.head(10))

cluster_sizes = cluster_sizes[cluster_sizes.index != -1] # no noise

max_cluster_size=cluster_sizes['count'].max()
print("max = ",max_cluster_size)

### MDS

In [ ]:
agg_func = {
    'DaysSince':['max','min'],
    'Longitude':['mean','max','min'],
    'Latitude':['mean','max','min'],
    'DayYear':['max','min']
}
st_aggregates = acc_evt_hsall.reset_index(drop=False)[['ClusterN', 'DaysSince',
                                                       'Longitude','Latitude', 'DayYear']].groupby(['ClusterN']).agg(agg_func)
# Flatten hierarchical column names
st_aggregates.columns = ["_".join(x) for x in st_aggregates.columns.ravel()]
# compute derived attributes: duration and bounding rectangle diagonal
st_aggregates['duration (days)']=st_aggregates['DaysSince_max']-st_aggregates['DaysSince_min']
for id,row in st_aggregates.iterrows():
    brd=kms_per_radian*great_circle2(row['Latitude_max'],row['Longitude_max'],
                                     row['Latitude_min'],row['Longitude_min'])
    #print('{}'.format(brd))
    #print(row['Latitude_max'],row['Longitude_max'],row['Latitude_min'],row['Longitude_min'])
    st_aggregates.at[id,'Bound_rect_diag(km)']=brd

    

clusters_data = st_aggregates.loc[st_aggregates.index!=-1,
                                  ['Latitude_mean','Longitude_mean',
                                   'DayYear_min','DayYear_max']]

scaler = MinMaxScaler()
clusters_data_scaled = scaler.fit_transform(clusters_data)

mds_ST = MDS(n_components = 2, random_state=110)
mds_ST.fit(clusters_data_scaled)
xy_mds_ST = mds_ST.fit_transform(clusters_data_scaled)

xmin_ST=xy_mds_ST[:,0].min() 
xmax_ST=xy_mds_ST[:,0].max()
ymin_ST=xy_mds_ST[:,1].min()
ymax_ST=xy_mds_ST[:,1].max()
#print(xmin_ST,xmax_ST,ymin_ST,ymax_ST)


fig, ax = plt.subplots(1,1,figsize=(18,16))

plt.style.use(('fivethirtyeight'))

plt.xlabel('Axis 1')
plt.ylabel('Axis 2')
plt.title('MDS projection (DBSCAN)')
colors = [(0,0,0)]

for i in range(len(xy_mds_ST)):
    j=np.where(cluster_sizes.index==clusters_data.index[i])[0][0]
    r=cluster_sizes.iat[j,0]/max_cluster_size
    size=50 + 300*r
    if r > 0.01:
        cl_text = str(clusters_data.index[i])+": "+str(cluster_sizes.iat[j,0])
    else:
        cl_text = ''
    ax.scatter(xy_mds_ST[i,0], xy_mds_ST[i,1], alpha = 1, s = size*2, 
                c=getColor(xy_mds_ST[i,0], xy_mds_ST[i,1],xmin_ST,xmax_ST,ymin_ST,ymax_ST))
    ax.text(xy_mds_ST[i,0]+0.0001*size, xy_mds_ST[i,1]+0.0001*size,
             cl_text, alpha = .6+.4*r, fontsize=18)

        
        
#ax.set_facecolor('white')
plt.grid(True)
plt.show()

#fig.savefig('mds_uk_hs_all_2dbscan.png')


In [ ]:


tile = basemap_to_tiles(basemaps.Esri.WorldStreetMap)


Esri_WorldStreetMap = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Street_Map/MapServer/tile/{z}/{y}/{x}'
Esri_Attribution = 'Tiles &copy; Esri &mdash; Source: Esri, DeLorme, NAVTEQ, USGS, Intermap, iPC, NRCAN, Esri Japan, METI, Esri China (Hong Kong), Esri (Thailand), TomTom, 2012'

#location=[54.384797 , -3.438721],zoom_start=6)
#location=[52.384797 , -3.438721],zoom_start=7)

m = folium.Map(tiles=Esri_WorldStreetMap, attr=Esri_Attribution, width='100%', height='100%', 
               location=[52.384797 , -3.438721],zoom_start=7)
# If you adjusted the notebook display width to be as wide as your screen, the map might get very big. 
#Adjust size as desired.
#m.fit_bounds([[lat_range[0], lon_range[0]], [lat_range[1], lon_range[1]]])
for id, row in acc_evt_hsall.iterrows():
    cluster_id = row[clust_id_col_name]
    if cluster_id != -1 and len(np.where(clusters_data.index==cluster_id)[0])>0:
        i=np.where(clusters_data.index==cluster_id)[0][0]
        if i<len(xy_mds_ST):
            folium.CircleMarker((row['Latitude'], row['Longitude']), radius=2, 
                        #color=clust_colors[cluster_id % len(clust_colors)], 
                        color=getColor(xy_mds_ST[i,0], xy_mds_ST[i,1],xmin_ST,xmax_ST,ymin_ST,ymax_ST),
                        fill=False, opacity=0.4,
                        popup='Cluster: {}'.format(cluster_id)).add_to(m)
            

m

#m.save(os.path.join('', 'all_accidents_lon.html'))

In [ ]:
#Save the map as an HTML file

delay=5

fn='testmap.html'
tmpurl='file://{path}/{mapfile}'.format(path=os.getcwd(),mapfile=fn)
m.save(fn)

#Open a browser window...
browser = webdriver.Chrome()
#..that displays the map...
browser.get(tmpurl)
#Give the map tiles some time to load
time.sleep(delay)
#Grab the screenshot
browser.save_screenshot('map_uk_hs_dbscan_06a.png')
#Close the browser
browser.quit()

### Preparation of Histograms of clusters

In [ ]:
for id, row in acc_evt_hsall.iterrows():
    cluster_id = row[clust_id_col_name]
    if cluster_id != -1 and len(np.where(clusters_data.index==cluster_id)[0])>0:
        i=np.where(clusters_data.index==cluster_id)[0][0]
        if i<len(xy_mds_ST):
            acc_evt_hsall.loc[id, 'Color']=getColor(xy_mds_ST[i,0], xy_mds_ST[i,1],
                                                xmin_ST,xmax_ST,ymin_ST,ymax_ST)

for id, row in acc_evt_hsmain.iterrows():
    cluster_id = row[clust_id_col_name]
    if cluster_id != -1 and len(np.where(clusters_data_main.index==cluster_id)[0])>0:
        i=np.where(clusters_data_main.index==cluster_id)[0][0]
        if i<len(xy_mds_ST_main):
            acc_evt_hsmain.loc[id, 'Color']=getColor(xy_mds_ST_main[i,0], xy_mds_ST_main[i,1],
                                                xmin_ST_main,xmax_ST_main,ymin_ST_main,ymax_ST_main)

In [ ]:
acc_evt_hsall[acc_evt_hsall.ClusterN==0]

### Histogram of UK and main clusters

In [ ]:
# UK clusters
clust_val = acc_evt_hsall[acc_evt_hsall.ClusterN!=-1].groupby(['ClusterN'])['Accident_Index'].count(
).reset_index().nlargest(20,'Accident_Index')['ClusterN'].values

acc = acc_evt_hsall[acc_evt_hsall.ClusterN.isin(clust_val)].groupby(['ClusterN',
               'Local_Authority_(District)'])['Accident_Index'].count().reset_index()

acc_bar = acc.loc[acc.groupby(['ClusterN','color'])['Accident_Index'].idxmax()].reset_index()
acc_bar

In [ ]:
# UK clusters
clust_val = acc_evt_hsall[acc_evt_hsall.ClusterN!=-1].groupby(['ClusterN'])['Accident_Index'].count(
).reset_index().nlargest(15,'Accident_Index')['ClusterN'].values

acc = acc_evt_hsall[acc_evt_hsall.ClusterN.isin(clust_val)].groupby(['ClusterN',
               'Local_Authority_(District)','Color'])['Accident_Index'].count().reset_index()

acc_bar = acc.loc[acc.groupby(['ClusterN','Color'])['Accident_Index'].idxmax()].reset_index()

# UK main cluster
clust_val = acc_evt_hsmain[acc_evt_hsmain.ClusterN!=-1].groupby(['ClusterN'])['Accident_Index'].count(
).reset_index().nlargest(15,'Accident_Index')['ClusterN'].values

acc = acc_evt_hsmain[acc_evt_hsmain.ClusterN.isin(clust_val)].groupby(['ClusterN',
               'Local_Authority_(District)','Color'])['Accident_Index'].count().reset_index()

acc_main_bar = acc.loc[acc.groupby(['ClusterN'])['Accident_Index'].idxmax()].reset_index()


fig, axs = plt.subplots(1,2,figsize=(32,13), sharey=True)

clrs = acc_bar.Color
clrs_main = acc_main_bar.Color
# Barplot
sns.barplot(data=acc_bar, x='Local_Authority_(District)', y='Accident_Index', palette=clrs, ax=axs[0])
axs[0].set_xticklabels(axs[0].get_xticklabels(),rotation=90,fontsize=22)
axs[0].yaxis.set_tick_params(labelsize=22)
axs[0].set_title('Most frequent LA of each cluster (top 15 in size) in the UK',fontsize=22)
axs[0].set_xlabel('')
axs[0].set_ylabel('')

sns.barplot(data=acc_main_bar, x='Local_Authority_(District)', y='Accident_Index', palette=clrs_main, ax=axs[1])
axs[1].set_xticklabels(axs[1].get_xticklabels(),rotation=90,fontsize=22)
axs[1].set_title('Most frequent LA of each cluster (top 15 in size) from main UK cluster',fontsize=22)
axs[1].set_xlabel('')
axs[1].set_ylabel('')


plt.tight_layout()
plt.show()

fig.savefig('fig12.png')

In [ ]:
display(acc_bar.Accident_Index.sum())
display(acc_main_bar.Accident_Index.sum())

In [ ]:
df3 = df[df.Accident_Severity!='Slight'].copy()

df3['Region'] = np.where(df3['Region']=='London','London',
                                             df3['Region'])

df3 = df3[df3['Region']!='London']

In [ ]:
#df3[df3['Region']=='London'].count()/df3[df3['Region']!='London'].count()

In [ ]:
# # UK clusters
# clust_val = acc_evt_hsall[acc_evt_hsall.ClusterN!=-1].groupby(['ClusterN'])['Accident_Index'].count(
# ).reset_index().nlargest(15,'Accident_Index')['ClusterN'].values

# acc = acc_evt_hsall[acc_evt_hsall.ClusterN.isin(clust_val)].groupby(['ClusterN',
#                'Local_Authority_(District)','Color'])['Accident_Index'].count().reset_index()

acc_2010 = df3[df3.Year==2010].groupby(['Region'])['Accident_Index'].count().reset_index().nlargest(10,'Accident_Index')
acc_2011 = df3[df3.Year==2011].groupby(['Region'])['Accident_Index'].count().reset_index().nlargest(10,'Accident_Index')
acc_2012 = df3[df3.Year==2012].groupby(['Region'])['Accident_Index'].count().reset_index().nlargest(10,'Accident_Index')
acc_2013 = df3[df3.Year==2013].groupby(['Region'])['Accident_Index'].count().reset_index().nlargest(10,'Accident_Index')
acc_2014 = df3[df3.Year==2014].groupby(['Region'])['Accident_Index'].count().reset_index().nlargest(10,'Accident_Index')
acc_2015 = df3[df3.Year==2015].groupby(['Region'])['Accident_Index'].count().reset_index().nlargest(10,'Accident_Index')
acc_2016 = df3[df3.Year==2016].groupby(['Region'])['Accident_Index'].count().reset_index().nlargest(10,'Accident_Index')
acc_2017 = df3[df3.Year==2017].groupby(['Region'])['Accident_Index'].count().reset_index().nlargest(10,'Accident_Index')


fig, axs = plt.subplots(8,1,figsize=(32,38))

 
# Barplot
sns.barplot(data=acc_2010, x='Region', y='Accident_Index', ax=axs[0])
sns.barplot(data=acc_2011, x='Region', y='Accident_Index', ax=axs[1])
sns.barplot(data=acc_2012, x='Region', y='Accident_Index', ax=axs[2])
sns.barplot(data=acc_2013, x='Region', y='Accident_Index', ax=axs[3])
sns.barplot(data=acc_2014, x='Region', y='Accident_Index', ax=axs[4])
sns.barplot(data=acc_2015, x='Region', y='Accident_Index', ax=axs[5])
sns.barplot(data=acc_2016, x='Region', y='Accident_Index', ax=axs[6])
sns.barplot(data=acc_2017, x='Region', y='Accident_Index', ax=axs[7])
axs[0].set_xticklabels(axs[0].get_xticklabels(),rotation=90,fontsize=18)
axs[1].set_xticklabels(axs[1].get_xticklabels(),rotation=90,fontsize=18)
axs[2].set_xticklabels(axs[2].get_xticklabels(),rotation=90,fontsize=18)
axs[3].set_xticklabels(axs[3].get_xticklabels(),rotation=90,fontsize=18)
axs[4].set_xticklabels(axs[4].get_xticklabels(),rotation=90,fontsize=18)
axs[5].set_xticklabels(axs[5].get_xticklabels(),rotation=90,fontsize=18)
axs[6].set_xticklabels(axs[6].get_xticklabels(),rotation=90,fontsize=18)
axs[7].set_xticklabels(axs[7].get_xticklabels(),rotation=90,fontsize=18)

# axs[0].yaxis.set_tick_params(labelsize=22)
# axs[0].set_title('Most frequent LA of each cluster (top 15 in size) in the UK',fontsize=22)
# axs[0].set_xlabel('')
# axs[0].set_ylabel('')

# sns.barplot(data=acc_main_bar, x='Local_Authority_(District)', y='Accident_Index', palette=clrs_main, ax=axs[1])
# axs[1].set_xticklabels(axs[1].get_xticklabels(),rotation=90,fontsize=22)
# axs[1].set_title('Most frequent LA of each cluster (top 15 in size) from main UK cluster',fontsize=22)
# axs[1].set_xlabel('')
# axs[1].set_ylabel('')


plt.tight_layout()
plt.show()

## Clustering of UK main cluster

In [ ]:
acc_evt_hsmain = acc_evt_hsall[acc_evt_hsall.ClusterN==0]
X_hsmain = acc_evt_hsall[acc_evt_hsall.ClusterN==0][['Longitude','Latitude']].values

In [ ]:
#X_hsall  = acc_evt_hsall[['Longitude','Latitude']].values
nearest_neighbors = NearestNeighbors(n_neighbors=10)
neighbors = nearest_neighbors.fit(X_hsmain)
distances, indices = neighbors.kneighbors(X_hsmain)
distances = np.sort(distances[:,9], axis=0)



i = np.arange(len(distances))
knee = KneeLocator(i, distances, S=1, curve='convex', direction='increasing', interp_method='polynomial')
# fig = plt.figure(figsize=(5, 5))
# knee.plot_knee()
# plt.xlabel("Points")
# plt.ylabel("Distance")

print(distances[knee.knee])
#plt.savefig("Distance_curve.png", dpi=300)

n_neighbours = 10

# represent points consistently as (lat, lon)
coords = acc_evt_hsmain[['Longitude','Latitude']]


# define epsilon as 1.5 kilometers, converted to radians for use by haversine
spatial_dist_max = distances[knee.knee] #/ kms_per_radian
#spatial_dist_max = 0.00006

clustered_ST = DBSCAN(eps=np.radians(distances[knee.knee]),
                      metric='haversine',min_samples=n_neighbours).fit(np.radians(coords))

print("Clustering finished!")

labels=clustered_ST.labels_
unique_labels=np.unique(clustered_ST.labels_)
print('Result: {} records in the noise, labelled as -1, and {} clusters labelled as 0..{}'.
      format(acc_evt_hsmain[labels==-1].shape[0], len(unique_labels)-1, len(unique_labels)-2))

#clustered
clust_id_col_name='ClusterN'
acc_evt_hsmain[clust_id_col_name]=labels

## Getting cluster sizes
cluster_sizes = acc_evt_hsmain[clust_id_col_name].value_counts().rename_axis('Cluster id').to_frame('count')
print("Cluster sizes:")
print(cluster_sizes.head(10))

cluster_sizes = cluster_sizes[cluster_sizes.index != -1] # no noise

max_cluster_size=cluster_sizes['count'].max()
print("max = ",max_cluster_size)

agg_func = {
    'DaysSince':['max','min'],
    'Longitude':['mean','max','min'],
    'Latitude':['mean','max','min'],
    'DayYear':['max','min']
}
st_aggregates = acc_evt_hsmain.reset_index(drop=False)[['ClusterN', 'DaysSince',
                                                       'Longitude','Latitude', 'DayYear']].groupby(['ClusterN']).agg(agg_func)
# Flatten hierarchical column names
st_aggregates.columns = ["_".join(x) for x in st_aggregates.columns.ravel()]
# compute derived attributes: duration and bounding rectangle diagonal
st_aggregates['duration (days)']=st_aggregates['DaysSince_max']-st_aggregates['DaysSince_min']
for id,row in st_aggregates.iterrows():
    brd=kms_per_radian*great_circle2(row['Latitude_max'],row['Longitude_max'],
                                     row['Latitude_min'],row['Longitude_min'])
    #print('{}'.format(brd))
    #print(row['Latitude_max'],row['Longitude_max'],row['Latitude_min'],row['Longitude_min'])
    st_aggregates.at[id,'Bound_rect_diag(km)']=brd

    

clusters_data_main = st_aggregates.loc[st_aggregates.index!=-1,
                                  ['Latitude_mean','Longitude_mean',
                                   'DayYear_min','DayYear_max']]

scaler = MinMaxScaler()
clusters_data_scaled = scaler.fit_transform(clusters_data_main)

mds_ST = MDS(n_components = 2, random_state=110)
mds_ST.fit(clusters_data_scaled)

xy_mds_ST_main = mds_ST.fit_transform(clusters_data_scaled)

xmin_ST_main=xy_mds_ST_main[:,0].min() 
xmax_ST_main=xy_mds_ST_main[:,0].max()
ymin_ST_main=xy_mds_ST_main[:,1].min()
ymax_ST_main=xy_mds_ST_main[:,1].max()
#print(xmin_ST,xmax_ST,ymin_ST,ymax_ST)



tile = basemap_to_tiles(basemaps.Esri.WorldStreetMap)


Esri_WorldStreetMap = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Street_Map/MapServer/tile/{z}/{y}/{x}'
Esri_Attribution = 'Tiles &copy; Esri &mdash; Source: Esri, DeLorme, NAVTEQ, USGS, Intermap, iPC, NRCAN, Esri Japan, METI, Esri China (Hong Kong), Esri (Thailand), TomTom, 2012'

#location=[54.384797 , -3.438721],zoom_start=6)
#location=[52.384797 , -3.438721],zoom_start=7)

m = folium.Map(tiles=Esri_WorldStreetMap, attr=Esri_Attribution, width='100%', height='100%', 
               location=[52.384797 , -3.438721],zoom_start=7)
# If you adjusted the notebook display width to be as wide as your screen, the map might get very big. 
#Adjust size as desired.
#m.fit_bounds([[lat_range[0], lon_range[0]], [lat_range[1], lon_range[1]]])
for id, row in acc_evt_hsmain.iterrows():
    cluster_id = row[clust_id_col_name]
    if cluster_id != -1 and len(np.where(clusters_data.index==cluster_id)[0])>0:
        i=np.where(clusters_data.index==cluster_id)[0][0]
        if i<len(xy_mds_ST_main):
            folium.CircleMarker((row['Latitude'], row['Longitude']), radius=2, 
                        #color=clust_colors[cluster_id % len(clust_colors)], 
                        color=getColor(xy_mds_ST_main[i,0], xy_mds_ST_main[i,1],
                                       xmin_ST_main,xmax_ST_main,ymin_ST_main,ymax_ST_main),
                        fill=False, opacity=0.4,
                        popup='Cluster: {}'.format(cluster_id)).add_to(m)
            

m


# delay=5

# fn='testmap.html'
# tmpurl='file://{path}/{mapfile}'.format(path=os.getcwd(),mapfile=fn)
# m.save(fn)

# #Open a browser window...
# browser = webdriver.Chrome()
# #..that displays the map...
# browser.get(tmpurl)
# #Give the map tiles some time to load
# time.sleep(delay)
# #Grab the screenshot
# browser.save_screenshot('map_ukclus1_hs_dbscan.png')
# #Close the browser
# browser.quit()

In [ ]:
clust_val = acc_evt_hsmain[acc_evt_hsmain.ClusterN!=-1].groupby(['ClusterN'])['Accident_Index'].count(
).reset_index().nlargest(20,'Accident_Index')['ClusterN'].values

acc = acc_evt_hsmain[acc_evt_hsmain.ClusterN.isin(clust_val)].groupby(['ClusterN',
               'Local_Authority_(District)'])['Accident_Index'].count().reset_index()

acc.loc[acc.groupby(['ClusterN'])['Accident_Index'].idxmax()]['Accident_Index'].sum()

In [ ]:
clust_val = acc_evt_hsall[acc_evt_hsall.ClusterN!=-1].groupby(['ClusterN'])['Accident_Index'].count(
).reset_index().nlargest(20,'Accident_Index')['ClusterN'].values

acc = acc_evt_hsall[acc_evt_hsall.ClusterN.isin(clust_val)].groupby(['ClusterN',
               'Local_Authority_(District)'])['Accident_Index'].count().reset_index()

acc.loc[acc.groupby(['ClusterN'])['Accident_Index'].idxmax()]

# 4.2 Clustering of London accidents

In [ ]:
london_borough = ['Hackney',
'Barking and Dagenham','Barnet',
'Camden',
'Hillingdon',
'Brent',
'Harrow',
'Croydon',
'Bexley',
'Ealing',
'Enfield',
'Hounslow',
'Hammersmith and Fulham',
'Havering',
'Bromley',
'Haringey',
'Islington',
'Kensington and Chelsea',
'City of London',
'Greenwich',
'Westminster',
'Lewisham',
'Southwark',
'Newham',
'Tower Hamlets',
'Redbridge',
'Sutton',
'Waltham Forest',
'Merton',
'Richmond upon Thames',
'Kingston upon Thames',
'Lambeth',
'Wandsworth']


#acc_london = acc_events[(acc_events.Year>=2013)] 
acc_evt_hs = df[
             #(df['Local_Authority_(District)'].isin(london_borough)) &
             (df.Region == 'London') &
             (df.Accident_Severity !='Slight') &
             #(df.Number_of_Casualties > 1) &
             (df.Year >= 2017)]

acc_evt_ls = df[
             #(df['Local_Authority_(District)'].isin(london_borough)) &
             (df.Region == 'London') &
             (df.Accident_Severity =='Slight') &
             #(df.Number_of_Casualties > 1) &
             (df.Year >= 2015)]

We would like first to analyse what would be the best choice for the parameter epsilon

In [ ]:
print('LS:',acc_evt_ls.shape)
print('HS:',acc_evt_hs.shape)

In [ ]:
print(X_hs[:,0].min(),X_hs[:,0].max())

In [ ]:
#X = acc_lon[['Longitude','Latitude']].values
X_hs  = acc_evt_hs[['Longitude','Latitude']].values
X_ls  = acc_evt_ls[['Longitude','Latitude']].values

## High severity accidents DBSCAN

### NearestNeighbors locator

In [ ]:

nearest_neighbors = NearestNeighbors(n_neighbors=9)
neighbors = nearest_neighbors.fit(X_hs)
distances, indices = neighbors.kneighbors(X_hs)
distances = np.sort(distances[:,8], axis=0)
fig = plt.figure(figsize=(5, 5))
plt.plot(distances)
plt.xlabel("Points")
plt.ylabel("Distance")


### Knee locator

In [ ]:

i = np.arange(len(distances))
knee = KneeLocator(i, distances, S=1, curve='convex', direction='increasing', interp_method='polynomial')
fig = plt.figure(figsize=(5, 5))
knee.plot_knee()
plt.xlabel("Points")
plt.ylabel("Distance")

print(distances[knee.knee])
plt.savefig("Distance_curve.png", dpi=300)

### Scatter plot using EPS

In [ ]:
# fig = plt.figure(figsize=(15, 10))
# fig.subplots_adjust(hspace=.5, wspace=.2)
# i = 1

# # represent points consistently as (lat, lon)
# coords = acc_eng[['Longitude','Latitude']][:100].values

# X=coords

# for x in range(10, 0, -1):
#     eps = 1/(10*(11-x))
#     db = DBSCAN(eps=eps, min_samples=7).fit(X)
#     core_samples_mask = np.zeros_like(db.labels_, dtype=bool)
#     core_samples_mask[db.core_sample_indices_] = True
#     labels = db.labels_
    
#     #print(eps)
#     ax = fig.add_subplot(2, 5, i)
#     #ax.text(1, 4, "eps = {}".format(round(eps, 3)), fontsize=25, ha="center")
#     sns.scatterplot(X[:,0], X[:,1], hue=["cluster-{}".format(x) for x in labels])
#     i += 1

# plt.tight_layout()
# plt.show()

In [ ]:
temporal_dist_max = 24
n_neighbours = 9

# represent points consistently as (lat, lon)
coords = acc_evt_hs[['Longitude','Latitude']]


# define epsilon as 1.5 kilometers, converted to radians for use by haversine
spatial_dist_max = distances[knee.knee] #/ kms_per_radian
#spatial_dist_max = 0.00006
#distances[knee.knee]
clustered_ST = DBSCAN(eps=np.radians(spatial_dist_max),
                      metric='haversine',min_samples=n_neighbours).fit(np.radians(coords))

print("Clustering finished!")

labels=clustered_ST.labels_
unique_labels=np.unique(clustered_ST.labels_)
print('Result: {} records in the noise, labelled as -1, and {} clusters labelled as 0..{}'.
      format(acc_evt_hs[labels==-1].shape[0], len(unique_labels)-1, len(unique_labels)-2))

#clustered
clust_id_col_name='ClusterN'
acc_evt_hs[clust_id_col_name]=labels

### MDS

In [ ]:

# days_week = [ 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']

# # Scale X
# X_hs = acc_evt_hs[['Longitude','Latitude','Hour', 'Number_of_Casualties']][:500].values
# scaler = StandardScaler()
# X_scaled = scaler.fit_transform(X_hs)

# mds = MDS(2,random_state=0)
# X_2d = mds.fit_transform(X_scaled)

# colors = ['red','yellow','green','blue','purple','orange','brown']
# plt.rcParams['figure.figsize'] = [12, 10]
# plt.rc('font', size=14)

# my_cmap = plt.get_cmap('BuPu')

# k=1
# for i in days_week:
    
#     subset = X_2d[np.array(acc_evt_hs[:500].Day_of_Week) == i]
  
#     x = [row[0] for row in subset]
#     y = [row[1] for row in subset]
#     plt.scatter(x,y,
#                 color=my_cmap(k / 7),label=i,alpha=0.7)
#     k+=1
#     #Day_of_Week
    
# plt.legend()
# plt.show()



# #fig.savefig('MDS HS - Latitude, Longitude, Hour and Number of Casualties.png')


### London Cluster (DBSCAN)

In [ ]:
## Getting cluster sizes
cluster_sizes = acc_evt_hs[clust_id_col_name].value_counts().rename_axis('Cluster id').to_frame('count')
print("Cluster sizes:")
print(cluster_sizes.head(10))

cluster_sizes = cluster_sizes[cluster_sizes.index != -1] # no noise

max_cluster_size=cluster_sizes['count'].max()
print("max = ",max_cluster_size)

In [ ]:
agg_func = {
    'DaysSince':['max','min'],
    'Longitude':['mean','max','min'],
    'Latitude':['mean','max','min']
}
st_aggregates = acc_evt_hs.reset_index(drop=False)[['ClusterN','DaysSince',
                                                'Longitude','Latitude']].groupby(['ClusterN']).agg(agg_func)
# Flatten hierarchical column names
st_aggregates.columns = ["_".join(x) for x in st_aggregates.columns.ravel()]
# compute derived attributes: duration and bounding rectangle diagonal
st_aggregates['duration (days)']=st_aggregates['DaysSince_max']-st_aggregates['DaysSince_min']
for id,row in st_aggregates.iterrows():
    brd=kms_per_radian*great_circle2(row['Latitude_max'],row['Longitude_max'],
                                     row['Latitude_min'],row['Longitude_min'])
    #print('{}'.format(brd))
    #print(row['Latitude_max'],row['Longitude_max'],row['Latitude_min'],row['Longitude_min'])
    st_aggregates.at[id,'Bound_rect_diag(km)']=brd

    

clusters_data = st_aggregates.loc[st_aggregates.index!=-1,
                                  ['Latitude_mean','Longitude_mean',
                                   'DaysSince_min','DaysSince_max']]

scaler = MinMaxScaler()
clusters_data_scaled = scaler.fit_transform(clusters_data)

mds_ST = MDS(n_components = 2, random_state=110)
mds_ST.fit(clusters_data_scaled)
xy_mds_ST = mds_ST.fit_transform(clusters_data_scaled)

xmin_ST=xy_mds_ST[:,0].min() 
xmax_ST=xy_mds_ST[:,0].max()
ymin_ST=xy_mds_ST[:,1].min()
ymax_ST=xy_mds_ST[:,1].max()
#print(xmin_ST,xmax_ST,ymin_ST,ymax_ST)


fig, ax = plt.subplots(1,1,figsize=(10,10))
plt.xlabel('Axis 1')
plt.ylabel('Axis 2')
plt.title('MDS projection (DBSCAN)')
colors = [(0,0,0)]
for i in range(len(xy_mds_ST)):
    j=np.where(cluster_sizes.index==clusters_data.index[i])[0][0]
    r=cluster_sizes.iat[j,0]/max_cluster_size
    size=50 + 300*r
    ax.scatter(xy_mds_ST[i,0], xy_mds_ST[i,1], alpha = .9, s = size, 
                c=getColor(xy_mds_ST[i,0], xy_mds_ST[i,1],xmin_ST,xmax_ST,ymin_ST,ymax_ST))
    ax.text(xy_mds_ST[i,0]+0.0001*size, xy_mds_ST[i,1]+0.0001*size,
             str(clusters_data.index[i])+": "+str(cluster_sizes.iat[j,0]), alpha = .6+.4*r)

plt.grid() 
plt.show()

#fig.savefig('mdslonhs_dbscan.png')


### Density Plot (High severity accidents in London 2015-2017)

In [ ]:

# tile = basemap_to_tiles(basemaps.Esri.WorldStreetMap)


# Esri_WorldStreetMap = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Street_Map/MapServer/tile/{z}/{y}/{x}'
# Esri_Attribution = 'Tiles &copy; Esri &mdash; Source: Esri, DeLorme, NAVTEQ, USGS, Intermap, iPC, NRCAN, Esri Japan, METI, Esri China (Hong Kong), Esri (Thailand), TomTom, 2012'

# lon_range = (-130.60, -52.75)
# lat_range = (17.13, 53.65)
# m = folium.Map(tiles=Esri_WorldStreetMap, attr=Esri_Attribution, width='100%', height='100%', 
#                location=[53.384797 , -3.438721],zoom_start=7) 
# # If you adjusted the notebook display width to be as wide as your screen, the map might get very big. 
# #Adjust size as desired.
# #m.fit_bounds([[lat_range[0], lon_range[0]], [lat_range[1], lon_range[1]]])

# folium.CircleMarker((acc_evt_hs['Latitude'], acc_evt_hs['Longitude']), radius=2, 
#             #color=clust_colors[cluster_id % len(clust_colors)], 
#             color='red', fill=False, opacity=.3).add_to(m)
# m

In [ ]:


tile = basemap_to_tiles(basemaps.Esri.WorldStreetMap)


Esri_WorldStreetMap = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Street_Map/MapServer/tile/{z}/{y}/{x}'
Esri_Attribution = 'Tiles &copy; Esri &mdash; Source: Esri, DeLorme, NAVTEQ, USGS, Intermap, iPC, NRCAN, Esri Japan, METI, Esri China (Hong Kong), Esri (Thailand), TomTom, 2012'


m = folium.Map(tiles=Esri_WorldStreetMap, attr=Esri_Attribution, width='100%', height='100%', 
               location=[51.504797 , -0.068721],zoom_start=10) 
# If you adjusted the notebook display width to be as wide as your screen, the map might get very big. 
#Adjust size as desired.
#m.fit_bounds([[lat_range[0], lon_range[0]], [lat_range[1], lon_range[1]]])
for id, row in acc_evt_hs.iterrows():
    cluster_id = row[clust_id_col_name]
    if cluster_id != -1 and len(np.where(clusters_data.index==cluster_id)[0])>0:
        i=np.where(clusters_data.index==cluster_id)[0][0]
        if i<len(xy_mds_ST):
            folium.CircleMarker((row['Latitude'], row['Longitude']), radius=2, 
                        #color=clust_colors[cluster_id % len(clust_colors)], 
                        color=getColor(xy_mds_ST[i,0], xy_mds_ST[i,1],xmin_ST,xmax_ST,ymin_ST,ymax_ST),
                        fill=False, opacity=1,
                        popup='Cluster: {}'.format(cluster_id)).add_to(m)
            

m

#m.save(os.path.join('', 'all_accidents_lon.html'))

In [ ]:
#Save the map as an HTML file
fn='testmap.html'
tmpurl='file://{path}/{mapfile}'.format(path=os.getcwd(),mapfile=fn)
m.save(fn)

#Open a browser window...
browser = webdriver.Chrome()
#..that displays the map...
browser.get(tmpurl)
#Give the map tiles some time to load
time.sleep(delay)
#Grab the screenshot
browser.save_screenshot('map_lon_hs_dbscan.png')
#Close the browser
browser.quit()

### Visualise the spatio-temporal distribution of the UK cluster members in a 3D view

In [ ]:
cluster_id

In [ ]:


fig = plt.figure(1, figsize=(24, 15))

plt.style.use(('fivethirtyeight'))

for id,row in acc_evt_hsall.iterrows():
    cluster_id = row[clust_id_col_name]
    if (cluster_id==-1) or len(np.where(clusters_data.index==cluster_id)[0])==0 :
        color='#000000'
    else:
        i=np.where(clusters_data.index==cluster_id)[0][0]
        color=getColor(xy_mds_ST[i,0], xy_mds_ST[i,1],xmin_ST,xmax_ST,ymin_ST,ymax_ST)
        #color=clust_colors[cluster_id % len(clust_colors)]
    acc_evt_hsall.at[id,'colors']=color
acc_evt_hsallnn = acc_evt_hsall[acc_evt_hsall[clust_id_col_name] != -1] # no noise 
ax = Axes3D(fig, rect=[0, 0, .95, 1], elev=35, azim=-55) # change parameters here for experimenting

fig.set_facecolor('white')
ax.set_facecolor('white')

ax.scatter(acc_evt_hsallnn['Longitude'], acc_evt_hsallnn['Latitude'], acc_evt_hsallnn['DayYear'], 
           c=acc_evt_hsallnn['colors'], alpha=0.8)
plt.show()

fig.savefig('axes3d_uk2_dbscan.png')

## Low severity accidents DBSCAN

### NearestNeighbors locator

In [ ]:

nearest_neighbors = NearestNeighbors(n_neighbors=7)
neighbors = nearest_neighbors.fit(X_ls)
distances, indices = neighbors.kneighbors(X_ls)
distances = np.sort(distances[:,6], axis=0)
fig = plt.figure(figsize=(5, 5))
plt.plot(distances)
plt.xlabel("Points")
plt.ylabel("Distance")


### Knee locator

In [ ]:

i = np.arange(len(distances))
knee = KneeLocator(i, distances, S=1, curve='convex', direction='increasing', interp_method='polynomial')
fig = plt.figure(figsize=(5, 5))
knee.plot_knee()
plt.xlabel("Points")
plt.ylabel("Distance")

print(distances[knee.knee])
plt.savefig("Distance_curve.png", dpi=300)

### Scatter plot using EPS

In [ ]:
# fig = plt.figure(figsize=(15, 10))
# fig.subplots_adjust(hspace=.5, wspace=.2)
# i = 1

# # represent points consistently as (lat, lon)
# coords = acc_eng[['Longitude','Latitude']][:100].values

# X=coords

# for x in range(10, 0, -1):
#     eps = 1/(10*(11-x))
#     db = DBSCAN(eps=eps, min_samples=7).fit(X)
#     core_samples_mask = np.zeros_like(db.labels_, dtype=bool)
#     core_samples_mask[db.core_sample_indices_] = True
#     labels = db.labels_
    
#     #print(eps)
#     ax = fig.add_subplot(2, 5, i)
#     #ax.text(1, 4, "eps = {}".format(round(eps, 3)), fontsize=25, ha="center")
#     sns.scatterplot(X[:,0], X[:,1], hue=["cluster-{}".format(x) for x in labels])
#     i += 1

# plt.tight_layout()
# plt.show()

In [ ]:
temporal_dist_max = 24
n_neighbours = 7

# represent points consistently as (lat, lon)
coords = acc_evt_ls[['Longitude','Latitude']]


# define epsilon as 1.5 kilometers, converted to radians for use by haversine
spatial_dist_max = distances[knee.knee] #/ kms_per_radian
#spatial_dist_max = 0.00006

clustered_ST = DBSCAN(eps=np.radians(distances[knee.knee]),
                      metric='haversine',min_samples=n_neighbours).fit(np.radians(coords))

print("Clustering finished!")

labels=clustered_ST.labels_
unique_labels=np.unique(clustered_ST.labels_)
print('Result: {} records in the noise, labelled as -1, and {} clusters labelled as 0..{}'.
      format(acc_evt_ls[labels==-1].shape[0], len(unique_labels)-1, len(unique_labels)-2))

#clustered
clust_id_col_name='ClusterN'
acc_evt_ls[clust_id_col_name]=labels

In [ ]:
## Getting cluster sizes
cluster_sizes = acc_evt_ls[clust_id_col_name].value_counts().rename_axis('Cluster id').to_frame('count')
print("Cluster sizes:")
print(cluster_sizes.head(10))

cluster_sizes = cluster_sizes[cluster_sizes.index != -1] # no noise

max_cluster_size=cluster_sizes['count'].max()
print("max = ",max_cluster_size)

In [ ]:
agg_func = {
    'DaysSince':['max','min'],
    'Longitude':['mean','max','min'],
    'Latitude':['mean','max','min']
}
st_aggregates = acc_evt_ls.reset_index(drop=False)[['ClusterN','DaysSince',
                                                'Longitude','Latitude']].groupby(['ClusterN']).agg(agg_func)
# Flatten hierarchical column names
st_aggregates.columns = ["_".join(x) for x in st_aggregates.columns.ravel()]
# compute derived attributes: duration and bounding rectangle diagonal
st_aggregates['duration (days)']=st_aggregates['DaysSince_max']-st_aggregates['DaysSince_min']
for id,row in st_aggregates.iterrows():
    brd=kms_per_radian*great_circle2(row['Latitude_max'],row['Longitude_max'],row['Latitude_min'],row['Longitude_min'])
    #print('{}'.format(brd))
    #print(row['Latitude_max'],row['Longitude_max'],row['Latitude_min'],row['Longitude_min'])
    st_aggregates.at[id,'Bound_rect_diag(km)']=brd

    

clusters_data = st_aggregates.loc[st_aggregates.index!=-1,
                                  ['Latitude_mean','Longitude_mean',
                                   'DaysSince_min','DaysSince_max']]

scaler = MinMaxScaler()
clusters_data_scaled = scaler.fit_transform(clusters_data)

mds_ST = MDS(n_components = 2, random_state=110)
mds_ST.fit(clusters_data_scaled)
xy_mds_ST = mds_ST.fit_transform(clusters_data_scaled)

xmin_ST=xy_mds_ST[:,0].min() 
xmax_ST=xy_mds_ST[:,0].max()
ymin_ST=xy_mds_ST[:,1].min()
ymax_ST=xy_mds_ST[:,1].max()
#print(xmin_ST,xmax_ST,ymin_ST,ymax_ST)


fig, ax = plt.subplots(1,1,figsize=(10,10))
plt.xlabel('Axis 1')
plt.ylabel('Axis 2')
plt.title('MDS projection (DBSCAN)')
colors = [(0,0,0)]
for i in range(len(xy_mds_ST)):
    j=np.where(cluster_sizes.index==clusters_data.index[i])[0][0]
    r=cluster_sizes.iat[j,0]/max_cluster_size
    size=50 + 300*r
    ax.scatter(xy_mds_ST[i,0], xy_mds_ST[i,1], alpha = .9, s = size, 
                c=getColor(xy_mds_ST[i,0], xy_mds_ST[i,1],xmin_ST,xmax_ST,ymin_ST,ymax_ST))
    ax.text(xy_mds_ST[i,0]+0.0001*size, xy_mds_ST[i,1]+0.0001*size,
             str(clusters_data.index[i])+": "+str(cluster_sizes.iat[j,0]), alpha = .6+.4*r)

plt.grid() 
plt.show()

fig.savefig('mdslonls_dbscan.png')


### Plot (All accidents in London 2015-2017)

In [ ]:
# def plotDot(point):
#     '''input: series that contains a numeric named latitude and a numeric named longitude
#     this function creates a CircleMarker and adds it to your this_map'''
#     htmlString = folium.Html(popopHTMLString(point), script=True)
#     folium.CircleMarker(location=[point.latitude, point.longitude],
#                         fill_color=color_dict[point[color_var]],   ####NOTE THE CHANGE IN THE COLOR
#                         radius=2,
#                         popup = folium.Popup(htmlString),
#                         weight=0).add_to(this_map)

 
    
# #use df.apply(,axis=1) to iterate through every row in your dataframe
# data.apply(plotDot, axis = 1)

In [ ]:

# tile = basemap_to_tiles(basemaps.Esri.WorldStreetMap)


# Esri_WorldStreetMap = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Street_Map/MapServer/tile/{z}/{y}/{x}'
# Esri_Attribution = 'Tiles &copy; Esri &mdash; Source: Esri, DeLorme, NAVTEQ, USGS, Intermap, iPC, NRCAN, Esri Japan, METI, Esri China (Hong Kong), Esri (Thailand), TomTom, 2012'

# lon_range = (-130.60, -52.75)
# lat_range = (17.13, 53.65)
# m = folium.Map(tiles=Esri_WorldStreetMap, attr=Esri_Attribution, width='100%', height='100%', 
#                location=[53.384797 , -3.438721],zoom_start=7) 
# # If you adjusted the notebook display width to be as wide as your screen, the map might get very big. 
# #Adjust size as desired.
# #m.fit_bounds([[lat_range[0], lon_range[0]], [lat_range[1], lon_range[1]]])

# folium.CircleMarker((acc_evt_ls['Latitude'], acc_evt_ls['Longitude']), radius=2, 
#             #color=clust_colors[cluster_id % len(clust_colors)], 
#             color='red', fill=False, opacity=.3).add_to(m)
# m

In [ ]:

tile = basemap_to_tiles(basemaps.Esri.WorldStreetMap)


Esri_WorldStreetMap = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Street_Map/MapServer/tile/{z}/{y}/{x}'
Esri_Attribution = 'Tiles &copy; Esri &mdash; Source: Esri, DeLorme, NAVTEQ, USGS, Intermap, iPC, NRCAN, Esri Japan, METI, Esri China (Hong Kong), Esri (Thailand), TomTom, 2012'


m = folium.Map(tiles=Esri_WorldStreetMap, attr=Esri_Attribution, width='100%', height='100%', 
               location=[51.504797 , -0.068721],zoom_start=10) 
# If you adjusted the notebook display width to be as wide as your screen, the map might get very big. 
#Adjust size as desired.
#m.fit_bounds([[lat_range[0], lon_range[0]], [lat_range[1], lon_range[1]]])
for id, row in acc_evt_ls.iterrows():
    cluster_id = row[clust_id_col_name]
    if cluster_id != -1 and len(np.where(clusters_data.index==cluster_id)[0])>0:
        i=np.where(clusters_data.index==cluster_id)[0][0]
        if i<len(xy_mds_ST):
            folium.CircleMarker((row['Latitude'], row['Longitude']), radius=2, 
                        #color=clust_colors[cluster_id % len(clust_colors)], 
                        color=getColor(xy_mds_ST[i,0], xy_mds_ST[i,1],xmin_ST,xmax_ST,ymin_ST,ymax_ST),
                        fill=False, opacity=1,
                        popup='Cluster: {}'.format(cluster_id)).add_to(m)
            

#m

#m.save(os.path.join('', 'all_accidents_lon.html'))

In [ ]:
#Save the map as an HTML file
fn='testmap.html'
tmpurl='file://{path}/{mapfile}'.format(path=os.getcwd(),mapfile=fn)
m.save(fn)

#Open a browser window...
browser = webdriver.Chrome()
#..that displays the map...
browser.get(tmpurl)
#Give the map tiles some time to load
time.sleep(delay)
#Grab the screenshot
browser.save_screenshot('map_lon_ls_dbscan.png')
#Close the browser
browser.quit()

### DBSCAN Light Condition

In [ ]:
acc_evt_hs

In [ ]:
acc_evt_day = acc_evt_hs[acc_evt_hs.Light_Conditions=='Daylight']
acc_evt_drk = acc_evt_hs[acc_evt_hs.Light_Conditions.str.contains('Darkness')]

X_day = acc_evt_day[['Longitude','Latitude']].values
X_drk = acc_evt_drk[['Longitude','Latitude']].values

In [ ]:
acc_evt_hs[acc_evt_hs.Light_Conditions.str.contains('Darkness')]['Hour'].value_counts()

### DBSCAN Daylight

In [ ]:
#df[df.Light_Conditions=='Daylight']['Hour'][:15].value_counts()

In [ ]:
nearest_neighbors = NearestNeighbors(n_neighbors=7)
neighbors = nearest_neighbors.fit(X_day)
distances, indices = neighbors.kneighbors(X_day)
distances = np.sort(distances[:,6], axis=0)
# fig = plt.figure(figsize=(5, 5))
# plt.plot(distances)
# plt.xlabel("Points")
# plt.ylabel("Distance")

i = np.arange(len(distances))
knee = KneeLocator(i, distances, S=1, curve='convex', direction='increasing', interp_method='polynomial')
# fig = plt.figure(figsize=(5, 5))
# knee.plot_knee()
# plt.xlabel("Points")
# plt.ylabel("Distance")

print(distances[knee.knee])
#plt.savefig("Distance_curve.png", dpi=300)

temporal_dist_max = 24
n_neighbours = 7

# represent points consistently as (lat, lon)
coords = acc_evt_day[['Longitude','Latitude']]


# define epsilon as 1.5 kilometers, converted to radians for use by haversine
spatial_dist_max = distances[knee.knee] #/ kms_per_radian
#spatial_dist_max = 0.00006

clustered_ST = DBSCAN(eps=np.radians(distances[knee.knee]),
                      metric='haversine',min_samples=n_neighbours).fit(np.radians(coords))

print("Clustering finished!")

labels=clustered_ST.labels_
unique_labels=np.unique(clustered_ST.labels_)
print('Result: {} records in the noise, labelled as -1, and {} clusters labelled as 0..{}'.
      format(acc_evt_day[labels==-1].shape[0], len(unique_labels)-1, len(unique_labels)-2))

#clustered
clust_id_col_name='ClusterN'
acc_evt_day[clust_id_col_name]=labels
## Getting cluster sizes
cluster_sizes = acc_evt_day[clust_id_col_name].value_counts().rename_axis('Cluster id').to_frame('count')
print("Cluster sizes:")
print(cluster_sizes.head(10))

cluster_sizes = cluster_sizes[cluster_sizes.index != -1] # no noise

max_cluster_size=cluster_sizes['count'].max()
print("max = ",max_cluster_size)


In [ ]:
agg_func = {
    'DaysSince':['max','min'],
    'Longitude':['mean','max','min'],
    'Latitude':['mean','max','min']
}
st_aggregates = acc_evt_day.reset_index(drop=False)[['ClusterN','DaysSince',
                                                'Longitude','Latitude']].groupby(['ClusterN']).agg(agg_func)
# Flatten hierarchical column names
st_aggregates.columns = ["_".join(x) for x in st_aggregates.columns.ravel()]
# compute derived attributes: duration and bounding rectangle diagonal
st_aggregates['duration (days)']=st_aggregates['DaysSince_max']-st_aggregates['DaysSince_min']
for id,row in st_aggregates.iterrows():
    brd=kms_per_radian*great_circle2(row['Latitude_max'],row['Longitude_max'],row['Latitude_min'],row['Longitude_min'])
    #print('{}'.format(brd))
    #print(row['Latitude_max'],row['Longitude_max'],row['Latitude_min'],row['Longitude_min'])
    st_aggregates.at[id,'Bound_rect_diag(km)']=brd

    

clusters_data = st_aggregates.loc[st_aggregates.index!=-1,
                                  ['Latitude_mean','Longitude_mean',
                                   'DaysSince_min','DaysSince_max']]

scaler = MinMaxScaler()
clusters_data_scaled = scaler.fit_transform(clusters_data)

mds_ST = MDS(n_components = 2, random_state=110)
mds_ST.fit(clusters_data_scaled)
xy_mds_ST = mds_ST.fit_transform(clusters_data_scaled)

xmin_ST=xy_mds_ST[:,0].min() 
xmax_ST=xy_mds_ST[:,0].max()
ymin_ST=xy_mds_ST[:,1].min()
ymax_ST=xy_mds_ST[:,1].max()
#print(xmin_ST,xmax_ST,ymin_ST,ymax_ST)

#fig.savefig('mdslonls_dbscan.png')


In [ ]:
tile = basemap_to_tiles(basemaps.Esri.WorldStreetMap)


Esri_WorldStreetMap = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Street_Map/MapServer/tile/{z}/{y}/{x}'
Esri_Attribution = 'Tiles &copy; Esri &mdash; Source: Esri, DeLorme, NAVTEQ, USGS, Intermap, iPC, NRCAN, Esri Japan, METI, Esri China (Hong Kong), Esri (Thailand), TomTom, 2012'


m = folium.Map(tiles=Esri_WorldStreetMap, attr=Esri_Attribution, width='100%', height='100%', 
               location=[51.504797 , -0.068721],zoom_start=10) 
# If you adjusted the notebook display width to be as wide as your screen, the map might get very big. 
#Adjust size as desired.
#m.fit_bounds([[lat_range[0], lon_range[0]], [lat_range[1], lon_range[1]]])
for id, row in acc_evt_day.iterrows():
    cluster_id = row[clust_id_col_name]
    if cluster_id != -1 and len(np.where(clusters_data.index==cluster_id)[0])>0:
        i=np.where(clusters_data.index==cluster_id)[0][0]
        if i<len(xy_mds_ST):
            folium.CircleMarker((row['Latitude'], row['Longitude']), radius=2, 
                        #color=clust_colors[cluster_id % len(clust_colors)], 
                        color=getColor(xy_mds_ST[i,0], xy_mds_ST[i,1],xmin_ST,xmax_ST,ymin_ST,ymax_ST),
                        fill=False, opacity=1,
                        popup='Cluster: {}'.format(cluster_id)).add_to(m)
            

#m

#m.save(os.path.join('', 'all_accidents_lon.html'))
#Save the map as an HTML file
# fn='testmap.html'
# tmpurl='file://{path}/{mapfile}'.format(path=os.getcwd(),mapfile=fn)
# m.save(fn)

# delay=5
# #Open a browser window...
# browser = webdriver.Chrome()
# #..that displays the map...
# browser.get(tmpurl)
# #Give the map tiles some time to load
# time.sleep(delay)
# #Grab the screenshot
# browser.save_screenshot('map_lon_day_dbscan.png')
# #Close the browser
# browser.quit()

### DBSCAN Darkness

In [ ]:
nearest_neighbors = NearestNeighbors(n_neighbors=7)
neighbors = nearest_neighbors.fit(X_drk)
distances, indices = neighbors.kneighbors(X_drk)
distances = np.sort(distances[:,6], axis=0)
#fig = plt.figure(figsize=(5, 5))
#plt.plot(distances)
#plt.xlabel("Points")
#plt.ylabel("Distance")

i = np.arange(len(distances))
knee = KneeLocator(i, distances, S=1, curve='convex', direction='increasing', interp_method='polynomial')
#fig = plt.figure(figsize=(5, 5))
#knee.plot_knee()
#plt.xlabel("Points")
#plt.ylabel("Distance")

print(distances[knee.knee])
#plt.savefig("Distance_curve.png", dpi=300)

temporal_dist_max = 24
n_neighbours = 7

# represent points consistently as (lat, lon)
coords = acc_evt_drk[['Longitude','Latitude']]


# define epsilon as 1.5 kilometers, converted to radians for use by haversine
spatial_dist_max = distances[knee.knee] #/ kms_per_radian
#spatial_dist_max = 0.00006

clustered_ST = DBSCAN(eps=np.radians(distances[knee.knee]),
                      metric='haversine',min_samples=n_neighbours).fit(np.radians(coords))

print("Clustering finished!")

labels=clustered_ST.labels_
unique_labels=np.unique(clustered_ST.labels_)
print('Result: {} records in the noise, labelled as -1, and {} clusters labelled as 0..{}'.
      format(acc_evt_drk[labels==-1].shape[0], len(unique_labels)-1, len(unique_labels)-2))

#clustered
clust_id_col_name='ClusterN'
acc_evt_drk[clust_id_col_name]=labels
## Getting cluster sizes
cluster_sizes = acc_evt_drk[clust_id_col_name].value_counts().rename_axis('Cluster id').to_frame('count')
print("Cluster sizes:")
print(cluster_sizes.head(10))

cluster_sizes = cluster_sizes[cluster_sizes.index != -1] # no noise

max_cluster_size=cluster_sizes['count'].max()
print("max = ",max_cluster_size)


In [ ]:
agg_func = {
    'DaysSince':['max','min'],
    'Longitude':['mean','max','min'],
    'Latitude':['mean','max','min']
}
st_aggregates = acc_evt_drk.reset_index(drop=False)[['ClusterN','DaysSince',
                                                'Longitude','Latitude']].groupby(['ClusterN']).agg(agg_func)
# Flatten hierarchical column names
st_aggregates.columns = ["_".join(x) for x in st_aggregates.columns.ravel()]
# compute derived attributes: duration and bounding rectangle diagonal
st_aggregates['duration (days)']=st_aggregates['DaysSince_max']-st_aggregates['DaysSince_min']
for id,row in st_aggregates.iterrows():
    brd=kms_per_radian*great_circle2(row['Latitude_max'],row['Longitude_max'],row['Latitude_min'],row['Longitude_min'])
    #print('{}'.format(brd))
    #print(row['Latitude_max'],row['Longitude_max'],row['Latitude_min'],row['Longitude_min'])
    st_aggregates.at[id,'Bound_rect_diag(km)']=brd

    

clusters_data = st_aggregates.loc[st_aggregates.index!=-1,
                                  ['Latitude_mean','Longitude_mean',
                                   'DaysSince_min','DaysSince_max']]

scaler = MinMaxScaler()
clusters_data_scaled = scaler.fit_transform(clusters_data)

mds_ST = MDS(n_components = 2, random_state=110)
mds_ST.fit(clusters_data_scaled)
xy_mds_ST = mds_ST.fit_transform(clusters_data_scaled)

xmin_ST=xy_mds_ST[:,0].min() 
xmax_ST=xy_mds_ST[:,0].max()
ymin_ST=xy_mds_ST[:,1].min()
ymax_ST=xy_mds_ST[:,1].max()
#print(xmin_ST,xmax_ST,ymin_ST,ymax_ST)

#fig.savefig('mdslonls_dbscan.png')


In [ ]:
tile = basemap_to_tiles(basemaps.Esri.WorldStreetMap)


Esri_WorldStreetMap = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Street_Map/MapServer/tile/{z}/{y}/{x}'
Esri_Attribution = 'Tiles &copy; Esri &mdash; Source: Esri, DeLorme, NAVTEQ, USGS, Intermap, iPC, NRCAN, Esri Japan, METI, Esri China (Hong Kong), Esri (Thailand), TomTom, 2012'


m = folium.Map(tiles=Esri_WorldStreetMap, attr=Esri_Attribution, width='100%', height='100%', 
               location=[51.504797 , -0.068721],zoom_start=10) 
# If you adjusted the notebook display width to be as wide as your screen, the map might get very big. 
#Adjust size as desired.
#m.fit_bounds([[lat_range[0], lon_range[0]], [lat_range[1], lon_range[1]]])
for id, row in acc_evt_drk.iterrows():
    cluster_id = row[clust_id_col_name]
    if cluster_id != -1 and len(np.where(clusters_data.index==cluster_id)[0])>0:
        i=np.where(clusters_data.index==cluster_id)[0][0]
        if i<len(xy_mds_ST):
            folium.CircleMarker((row['Latitude'], row['Longitude']), radius=2, 
                        #color=clust_colors[cluster_id % len(clust_colors)], 
                        color=getColor(xy_mds_ST[i,0], xy_mds_ST[i,1],xmin_ST,xmax_ST,ymin_ST,ymax_ST),
                        fill=False, opacity=1,
                        popup='Cluster: {}'.format(cluster_id)).add_to(m)
            

m

# #m.save(os.path.join('', 'all_accidents_lon.html'))
# #Save the map as an HTML file
# fn='testmap.html'
# tmpurl='file://{path}/{mapfile}'.format(path=os.getcwd(),mapfile=fn)
# m.save(fn)

# delay=5
# #Open a browser window...
# browser = webdriver.Chrome()
# #..that displays the map...
# browser.get(tmpurl)
# #Give the map tiles some time to load
# time.sleep(delay)
# #Grab the screenshot
# browser.save_screenshot('map_lon_drk_dbscan.png')
# #Close the browser
# browser.quit()

## K-DBSCAN

In [ ]:
from sklearn.datasets import load_iris
from sklearn.metrics.pairwise import haversine_distances
X = load_iris().data
#X

In [ ]:
alg1_labels

In [ ]:
acc_evt_hdb_hs = acc_evt_hs.copy()
acc_evt_hdb_ls = acc_evt_ls.copy()

In [ ]:
# KDBSCAN
#import kdbscan
import importlib
importlib.reload(kdbscan)
from kdbscan import KDBSCAN

In [ ]:
acc_evt_hdb_hs[acc_evt_hdb_hs.Year==2017].shape

In [ ]:
alg1 = KDBSCAN(h=0.1,t=0.18,metric=cosine)

#X = load_iris().data

kde = alg1.fit(acc_evt_hdb_hs[acc_evt_hdb_hs.Year==2017][['Latitude','Longitude']].values,return_kde = True)
#kde = alg1.fit(X[:,1:3],return_kde = True)

alg1_labels = alg1.labels_
c1, c2, kept = kdbscan.plot_kdbscan_results(kde)

In [ ]:
alg1 = KDBSCAN(h=0.1,t=0.18,metric=cosine)

#X = load_iris().data

kde = alg1.fit(acc_evt_hdb_hs[acc_evt_hdb_hs.Year==2017][['Latitude','Longitude']].values,return_kde = True)
#kde = alg1.fit(X[:,1:3],return_kde = True)

alg1_labels = alg1.labels_
c1, c2, kept = kdbscan.plot_kdbscan_results(kde)

In [ ]:
c1, c2, kept 

In [ ]:
data_geo

In [ ]:
filtered_boundaries = gb_boundaries[(gb_boundaries.Year.isin(['2017']))]
#&                                     (gb_boundaries.Region.isin(['Scotland','East']))]
bound_val = filtered_boundaries[(filtered_boundaries.Region=='London')
                               #& (filtered_boundaries.geo_code=='E09000021')
                               ].reset_index()
bound_val[bound_val.lad20nm.str.contains('Lond')]

In [ ]:
import requests
import json
url='https://github.com/martinjc/UK-GeoJSON/blob/master/json/administrative/eng/lad.json?raw=true'
r = requests.get(url)

with open("lad.json", "wb") as code:
    code.write(r.content)

#Free up memory...
r=None

with open('lad.json', 'r') as output:
    boundaries = json.load(output)

In [ ]:
data_geo

In [ ]:
bound_val.head(1)

In [ ]:
valid_boroughs = ['Westminster',
'Haringey',
'Ealing',
'Croydon',
'Barnet',
'Barking and Dagenham',
'Hounslow',
'Kingston upon Thames',
'Barnet',
'Greenwich',
'Havering',
'Bexley',
'Redbridge']

In [ ]:
data_geo

In [ ]:
tile = basemap_to_tiles(basemaps.Esri.WorldStreetMap)


Esri_WorldStreetMap = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Street_Map/MapServer/tile/{z}/{y}/{x}'
Esri_Attribution = 'Tiles &copy; Esri &mdash; Source: Esri, DeLorme, NAVTEQ, USGS, Intermap, iPC, NRCAN, Esri Japan, METI, Esri China (Hong Kong), Esri (Thailand), TomTom, 2012'

#f = folium.Figure(width=1510, height=810)
m = folium.Map(tiles=white_tile,attr='white tile', 
               width='100%', height='100%',
               location=[51.504797 , -0.068721],zoom_start=10)


folium.Choropleth(
    geo_data=boundaries,
    data=bound_val,
    columns=['geo_code', 'High_Severity'],
    key_on='feature.properties.LAD13CD',
    fill_color='YlGnBu',
    fill_opacity=0.8,
    line_opacity=0.4,
    nan_fill_color='white',
    nan_fill_opacity=1,
    legend_name='High severity Accidents in 2017',
    
).add_to(m)

vb_idx=0
for idx in range(c1.shape[0]):
    if kept[idx]:
        folium.CircleMarker((c2[idx], c1[idx]), radius=4, 
                    color='magenta',
                    fill=False, opacity=1).add_to(m)
        if valid_boroughs[vb_idx] == 'Greenwich':
            folium.map.Marker(
                        (c2[idx], c1[idx]),
                        icon=DivIcon(
                            icon_size=(150,36),
                            icon_anchor=(0,0),
                            html='<div style="left: -50px; position: relative; font-size: 12pt"><b>%s</b></div>' % str(valid_boroughs[vb_idx]),
                            )
                        ).add_to(m)
        else:
            folium.map.Marker(
                        (c2[idx], c1[idx]),
                        icon=DivIcon(
                            icon_size=(150,36),
                            icon_anchor=(0,0),
                            html='<div style="font-size: 12pt"><b>%s</b></div>' % str(valid_boroughs[vb_idx]),
                            )
                        ).add_to(m)
        vb_idx+=1

        
folium.TileLayer('openstreetmap').add_to(m)

#m



fn='testmap.html'
tmpurl='file://{path}/{mapfile}'.format(path=os.getcwd(),mapfile=fn)
m.save(fn)

delay=3
#Open a browser window...
browser = webdriver.Chrome()
#..that displays the map...
browser.get(tmpurl)
#Give the map tiles some time to load
time.sleep(delay)
#Grab the screenshot
browser.save_screenshot('map_lon_hdbscan.png')
#Close the browser
browser.quit()

In [ ]:
c1

## Meanshift implementation

In [ ]:



# define the model
ms_model = MeanShift()
# fit model and predict clusters
yhat = ms_model.fit_predict(X_hs)
# retrieve unique clusters
clusters = unique(yhat)
# create scatter plot for samples from each cluster
for cluster in clusters:
    # get row indexes for samples with this cluster
    row_ix = where(yhat == cluster)
    # create scatter of these samples
    plt.scatter(X_hs[row_ix, 0], X_hs[row_ix, 1])
# show the plot
pyplot.show()

In [ ]:
# for idx in range(centroids.shape[0]):
#     if kept[idx]:
#         ax.plot(c1[idx], c2[idx], 'm^', markersize=20)
#     else:
#         ax.plot(c1[idx], c2[idx], 'k^', markersize=18)  	

In [ ]:
X_ran = df[((df.Region == 'London') | (df.Region == 'Wales')) &
             (df.Accident_Severity !='Slight') &
             (df.Year >= 2017)][['Longitude','Latitude']].values

In [ ]:
tile = basemap_to_tiles(basemaps.Esri.WorldStreetMap)


Esri_WorldStreetMap = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Street_Map/MapServer/tile/{z}/{y}/{x}'
Esri_Attribution = 'Tiles &copy; Esri &mdash; Source: Esri, DeLorme, NAVTEQ, USGS, Intermap, iPC, NRCAN, Esri Japan, METI, Esri China (Hong Kong), Esri (Thailand), TomTom, 2012'


m = folium.Map(tiles=Esri_WorldStreetMap, attr=Esri_Attribution, width='100%', height='100%', 
               location=[51.504797 , -0.068721],zoom_start=10) 
# If you adjusted the notebook display width to be as wide as your screen, the map might get very big. 
#Adjust size as desired.
#m.fit_bounds([[lat_range[0], lon_range[0]], [lat_range[1], lon_range[1]]])
for id, row in acc_evt_hs.iterrows():
    cluster_id = row[clust_id_col_name]
    if cluster_id != -1 and len(np.where(clusters_data.index==cluster_id)[0])>0:
        i=np.where(clusters_data.index==cluster_id)[0][0]
        if i<len(xy_mds_ST):
            folium.CircleMarker((row['Latitude'], row['Longitude']), radius=2, 
                        #color=clust_colors[cluster_id % len(clust_colors)], 
                        color=getColor(xy_mds_ST[i,0], xy_mds_ST[i,1],xmin_ST,xmax_ST,ymin_ST,ymax_ST),
                        fill=False, opacity=0.7,
                        popup='Cluster: {}'.format(cluster_id)).add_to(m)

m

### Spatial-temp DBSCAN

### Exploring spatial variation in the Number of accidents per LA

##### Read input files

In [ ]:
df[df.Region=='London'][['Local_Authority_(District)','Area_Code']].value_counts()

In [ ]:
df_a2013 = df_group_altair(df,2013)
df_a2014 = df_group_altair(df,2014)
df_a2015 = df_group_altair(df,2015)
df_a2016 = df_group_altair(df,2016)
df_a2017 = df_group_altair(df,2017)

df_alt = pd.concat([df_a2013, 
                           df_a2014, 
                           df_a2015, 
                           df_a2016, 
                           df_a2017], axis=0)
df_alt = df_alt.fillna(0)

In [ ]:

# uk_region = uk_region.append({'Area_Code': 'E09000001', 'Region': 'London', 'Area': 'City of London'}, ignore_index=True)
# a = uk_region[['Area_Code','Region','Area']]
# a[a.Area_Code.str.contains('E0900000')]

In [ ]:
SpaceTimeDistance

In [ ]:
#gb_boundaries = pd.merge(gb,referendum_data,left_on='geo_code', right_on='Area_Code', how='inner')
gb_boundaries = pd.merge(gb,df_alt,left_on='geo_code', right_on='Area_Code', how='inner')
gb_boundaries = pd.merge(gb_boundaries,uk_region[['Area_Code','Region','Area']],left_on='geo_code', 
                         right_on='Area_Code', how='inner')

gb_boundaries.rename(columns={('Accident_Index', 'Fatal'): 'Accidents_Fatal'}, inplace=True)
gb_boundaries.rename(columns={('Accident_Index', 'Serious'): 'Accidents_Serious'}, inplace=True)
gb_boundaries.rename(columns={('Accident_Index', 'Slight'): 'Accidents_Slight'}, inplace=True)
gb_boundaries.rename(columns={('Casualties', ''): 'Casualties'}, inplace=True)
gb_boundaries.rename(columns={('Year', ''): 'Year'}, inplace=True)
gb_boundaries['High_Severity'] = gb_boundaries['Accidents_Serious'] + gb_boundaries['Accidents_Fatal']
gb_boundaries['Year'] = gb_boundaries['Year'].apply(str)

In [ ]:


altCentroid = pd.DataFrame({'Longitude': c1, 'Latitude': c2, 'Valid': kept})
altCentroid[altCentroid.Valid==True]

In [ ]:
filtered_boundaries = gb_boundaries[(gb_boundaries.Year.isin(['2017']))]
#&                                     (gb_boundaries.Region.isin(['Scotland','East']))]

for idx in range(c1.shape[0]):
    if kept[idx]:
        folium.CircleMarker((c2[idx], c1[idx]), radius=4, 
                    #color=clust_colors[cluster_id % len(clust_colors)], 
                    color='black',
                    fill=False, opacity=1).add_to(m)

In [ ]:
data_geo = alt.InlineData(values = filtered_boundaries
                          [filtered_boundaries.Region=='London'].to_json(), #geopandas to geojson string
                       format = alt.DataFormat(property='features',type='json'))

base = alt.Chart(data_geo).properties(
    projection={'type': 'identity','reflectY': True},
    width=500,
    height=600
)

# chart13 = base.mark_geoshape(strokeWidth=1,stroke='lightgray',strokeOpacity=0.2
# ).encode(
#     color=alt.Color('properties.Accidents_Fatal:Q', scale=alt.Scale(scheme='blueorange')),
#     tooltip=['properties.Area:N','properties.Accidents_Fatal:Q','properties.Region:N'],
# ).transform_filter(
#     ('properties.Area:N' == 2013)
# )

chartpoints = alt.Chart(altCentroid[altCentroid.Valid==True]).mark_circle().encode(
    longitude='Longitude:Q',
    latitude='Latitude:Q',
    size=alt.value(10)
)

chart17 = base.mark_geoshape(strokeWidth=1,stroke='lightgray',strokeOpacity=0.2
).encode(
    color=alt.Color('properties.High_Severity:Q', scale=alt.Scale(scheme='blueorange')),
    tooltip=['properties.Area:N','properties.High_Severity:Q','properties.Region:N'],
)




chart17 + chartpoints

In [ ]:
data_geo = alt.InlineData(values = filtered_boundaries[filtered_boundaries['Region'] == 'London'].to_json(), #geopandas to geojson string
                       format = alt.DataFormat(property='features',type='json'))

alt.Chart(data_geo).mark_geoshape(strokeWidth=1,stroke='lightgray',strokeOpacity=0.2
).encode(
    color=alt.Color('properties.High_Severity:Q', scale=alt.Scale(scheme='blueorange')),
    tooltip=['properties.Area:N','properties.Casualties:Q','properties.Region:N']
).properties(
    projection={'type': 'identity','reflectY': True},
    width=500,
    height=600
)


In [ ]:
tile = basemap_to_tiles(basemaps.Esri.WorldStreetMap)


Esri_WorldStreetMap = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Street_Map/MapServer/tile/{z}/{y}/{x}'
Esri_Attribution = 'Tiles &copy; Esri &mdash; Source: Esri, DeLorme, NAVTEQ, USGS, Intermap, iPC, NRCAN, Esri Japan, METI, Esri China (Hong Kong), Esri (Thailand), TomTom, 2012'

# Create a white image of 4 pixels, and embed it in a url.
white_tile = branca.utilities.image_to_url([[1, 1], [1, 1]])

m = folium.Map(tiles=white_tile,attr='white tile',
               dragging=True, width='100%', height='100%', 
               location=[51.504797 , -0.068721],zoom_start=10) 
# If you adjusted the notebook display width to be as wide as your screen, the map might get very big. 
#Adjust size as desired.

g = folium.Choropleth(
    geo_data=gb_boundaries,
    data=gb,
    columns=['High_Severity'],
    key_on='properties.High_Severity',
    fill_color='YlGn',
    fill_opacity=0.7,
    line_opacity=0.4,
    legend_name='Data Points',
    highlight=True,
).add_to(m)

# for idx in range(c1.shape[0]):
#     if kept[idx]:
#         folium.CircleMarker((c2[idx], c1[idx]), radius=4, 
#                     #color=clust_colors[cluster_id % len(clust_colors)], 
#                     color='black',
#                     fill=False, opacity=1).add_to(m)


m

#m.save(os.path.join('', 'all_accidents_lon.html'))
#Save the map as an HTML file
# fn='testmap.html'
# tmpurl='file://{path}/{mapfile}'.format(path=os.getcwd(),mapfile=fn)
# m.save(fn)

# delay=5
# #Open a browser window...
# browser = webdriver.Chrome()
# #..that displays the map...
# browser.get(tmpurl)
# #Give the map tiles some time to load
# time.sleep(delay)
# #Grab the screenshot
# browser.save_screenshot('map_lon_drk_dbscan.png')
# #Close the browser
# browser.quit()

In [ ]:
c1

### Weather distribution per Year (London)

In [ ]:
df_weather = acc_lon.groupby(['Year','Weather_Conditions']).count()
df_weather = df_weather.sort_values(['Accident_Index'],ascending=False).reset_index()
df_weather.head(1)

In [ ]:
gb_boundaries

In [ ]:

# plot barh chart with index as x values

g = sns.catplot(x='Weather_Conditions', y='Accident_Index', 
            col="Year", col_wrap=5,
            kind='bar', data=df_weather, height=18, aspect= 0.5)

g.set_xticklabels(rotation=90,size = 30)
g.set_yticklabels(size = 100)
g.set_titles(size = 32)


plt.tight_layout()
plt.show()

## K-means

In [ ]:
X = acc_evt[["Latitude","Longitude"]]
map_shape = (4,4)
## scale data
scaler = preprocessing.StandardScaler()
X_preprocessed = scaler.fit_transform(X.values)
## clustering
model = minisom.MiniSom(x=map_shape[0], y=map_shape[1], 
                        input_len=X.shape[1])
model.train_batch(X_preprocessed, num_iteration=100, verbose=False)
## build output dataframe
dtf_X = X.copy()
dtf_X["cluster"] = np.ravel_multi_index(np.array(
      [model.winner(x) for x in X_preprocessed]).T, dims=map_shape)
## find real centroids
#cluster_centers = np.array([vec for center in model.get_weights() 
#                            for vec in center])

#cluster_centers, _ = kmeans(X_preprocessed, 5) 

closest, distances = scipy.cluster.vq.vq(cluster_centers, X_preprocessed)

dtf_X["centroids"] = 0

for i in closest:
    dtf_X["centroids"].iloc[i] = 1
## add clustering info to the original dataset
acc_evt[["cluster","centroids"]] = dtf_X[["cluster","centroids"]]
## plot
k = acc_evt["cluster"].nunique()
fig, ax = plt.subplots(figsize=(14,8))
sns.scatterplot(x="Latitude", y="Longitude", data=acc_evt, 
                palette=sns.color_palette("bright",k),
                hue='cluster', size="centroids", size_order=[1,0],
                legend="brief", ax=ax).set_title('Clustering (k='+str(k)+')')
th_centroids = scaler.inverse_transform(cluster_centers)
ax.scatter(th_centroids[:,0], th_centroids[:,1], s=50, c='black', 
           marker="x")

ax.legend(loc='center left', bbox_to_anchor=(1.25, 0.5), ncol=1)
plt.tight_layout()

In [ ]:
gb_boundaries.head(1)

In [ ]:
X_preprocessed

In [ ]:

# Declaring variables for use
distortions = []
# Populating distortions for various clusters

num_clusters = range(2, 15)

for i in num_clusters:
    centroids, distortion = kmeans(X_preprocessed, i)
    distortions.append(distortion)
    
# Plotting elbow plot data
elbow_plot_data = pd.DataFrame({'num_clusters': num_clusters,
'distortions': distortions})
sns.lineplot(x='num_clusters', y='distortions',
data = elbow_plot_data)
plt.show()

In [ ]:
x, y = "Latitude", "Longitude"
color = "cluster"
size = "Number_of_Casualties"
popup1 = "Number_of_Casualties"
popup2 = "Number_of_Vehicles"
marker = "centroids"
data = acc_evt.copy()
## create color column
lst_elements = sorted(list(acc_evt[color].unique()))
lst_colors = ['#%06X' % np.random.randint(0, 0xFFFFFF) for i in 
              range(len(lst_elements))]
data["color"] = data[color].apply(lambda x: 
                lst_colors[lst_elements.index(x)])
## create size column (scaled)
scaler = preprocessing.MinMaxScaler(feature_range=(3,15))
data["size"] = scaler.fit_transform(
               data[size].values.reshape(-1,1)).reshape(-1)
## initialize the map with the starting location
map_ = folium.Map(location=[51.584797 , 0.138721], tiles="cartodbpositron",
                  zoom_start=9.8)
## add points
data.apply(lambda row: folium.CircleMarker(
           location=[row[x],row[y]], popup='Number of Casualties:' + str(row[popup1]) + '<br>' + 
           'Number of Vehicles:' + str(row[popup2]),
           color=row["color"], fill=True,
           radius=row["size"]).add_to(map_), axis=1)
## add html legend
legend_html = """<div style="position:fixed; bottom:10px; left:10px; border:2px solid black; z-index:9999; font-size:14px;">&nbsp;<b>"""+color+""":</b><br>"""
for i in lst_elements:
     legend_html = legend_html+"""&nbsp;<i class="fa fa-circle 
     fa-1x" style="color:"""+lst_colors[lst_elements.index(i)]+"""">
     </i>&nbsp;"""+str(i)+"""<br>"""
legend_html = legend_html+"""</div>"""
map_.get_root().html.add_child(folium.Element(legend_html))
## add centroids marker
lst_elements = sorted(list(acc_evt[marker].unique()))
# data[data[marker]==1].apply(lambda row: 
#            folium.Marker(location=[row[x],row[y]], 
#            popup=row[marker], draggable=False,          
#            icon=folium.Icon(color="black")).add_to(map_), axis=1)
## plot the map
map_

In [ ]:
ssd = []
for i in range(2, 26):
    km = MiniBatchKMeans(n_clusters=i)
    km.fit_predict(X)
    ssd.append(km.inertia_)

### Multiple casualties

In [ ]:
dtf_X

In [ ]:
X = acc_lon[acc_lon.Number_of_Casualties > 1][["Latitude","Longitude"]]
map_shape = (4,4)
## scale data
scaler = preprocessing.StandardScaler()
X_preprocessed = scaler.fit_transform(X.values)
## clustering
model = minisom.MiniSom(x=map_shape[0], y=map_shape[1], 
                        input_len=X.shape[1])
model.train_batch(X_preprocessed, num_iteration=100, verbose=False)
## build output dataframe
dtf_X = X.copy()
dtf_X["cluster"] = np.ravel_multi_index(np.array(
      [model.winner(x) for x in X_preprocessed]).T, dims=map_shape)
## find real centroids
cluster_centers = np.array([vec for center in model.get_weights() 
                            for vec in center])


closest, distances = scipy.cluster.vq.vq(cluster_centers, X_preprocessed)

dtf_X["centroids"] = 0
for i in closest:
    dtf_X["centroids"].iloc[i] = 1
## add clustering info to the original dataset
acc_lon[["cluster","centroids"]] = dtf_X[["cluster","centroids"]]
## plot
k = acc_lon["cluster"].nunique()

lst_elements = sorted(list(acc_lon[marker].unique()))

In [ ]:
x, y = "Latitude", "Longitude"
color = "cluster"
size = "Number_of_Casualties"
popup1 = "Number_of_Casualties"
popup2 = "Number_of_Vehicles"
marker = "centroids"
data = acc_lon[acc_lon.Number_of_Casualties > 1].copy()
## create color column
lst_elements = sorted(list(acc_lon[color].unique()))
lst_colors = ['#%06X' % np.random.randint(0, 0xFFFFFF) for i in 
              range(len(lst_elements))]
data["color"] = data[color].apply(lambda x: 
                lst_colors[lst_elements.index(x)])
## create size column (scaled)
scaler = preprocessing.MinMaxScaler(feature_range=(3,15))
data["size"] = scaler.fit_transform(
               data[size].values.reshape(-1,1)).reshape(-1)
## initialize the map with the starting location
map_ = folium.Map(location=[51.584797 , 0.138721], tiles="cartodbpositron",
                  zoom_start=9.8)
## add points
data.apply(lambda row: folium.CircleMarker(
           location=[row[x],row[y]], popup='Number of Casualties:' + str(row[popup1]) + '<br>' + 
           'Number of Vehicles:' + str(row[popup2]),
           color=row["color"], fill=True,
           radius=row["size"]).add_to(map_), axis=1)
## add html legend
legend_html = """<div style="position:fixed; bottom:10px; left:10px; border:2px solid black; z-index:9999; font-size:14px;">&nbsp;<b>"""+color+""":</b><br>"""
for i in lst_elements:
     legend_html = legend_html+"""&nbsp;<i class="fa fa-circle 
     fa-1x" style="color:"""+lst_colors[lst_elements.index(i)]+"""">
     </i>&nbsp;"""+str(i)+"""<br>"""
legend_html = legend_html+"""</div>"""
map_.get_root().html.add_child(folium.Element(legend_html))
## add centroids marker
lst_elements = sorted(list(acc_lon[marker].unique()))

## plot the map
map_

### Perceptually uniform colour space

In [ ]:


# first draw a circle in the cylindrical JCh color space. 
# the third channel is hue in degrees. First is lightness and the second chroma
color_circle = np.ones((256,3))*60
color_circle[:,1] = np.ones((256))*45
color_circle[:,2] = np.arange(0,360,360/256)
color_circle_rgb = cspace_convert(color_circle, "JCh","sRGB1")

cm = col.ListedColormap(color_circle_rgb)

##### generate data grid like in above
N=256
x = np.linspace(-1,1,N)
y = np.linspace(-1,1,N)
z = np.zeros((len(y),len(x))) # make cartesian grid
for ii in range(len(y)): 
    z[ii] = np.arctan2(y[ii],x) # simple angular function

fig, ax = plt.subplots(figsize=(12,8))
ax = plt.gca()
pmesh = ax.pcolormesh(x, y, (z/np.pi)*1, 
    cmap = cm, vmin=-1, vmax=1)
plt.axis([x.min(), x.max(), y.min(), y.max()])
cbar = fig.colorbar(pmesh)
cbar.ax.set_ylabel('Phase [pi]')

print(np.arctan2(0.5,0)/np.pi)
plt.tight_layout()
plt.show()

## OPTICS

In [ ]:
n_points_per_cluster = 250


clust = OPTICS(min_samples=50, xi=.25, min_cluster_size=.05)

# Run the fit
clust.fit(X_hsall)

space = np.arange(len(X_hsall))
reachability = clust.reachability_[clust.ordering_]
labels = clust.labels_[clust.ordering_]

plt.figure(figsize=(10, 7))
G = gridspec.GridSpec(2, 3)
ax1 = plt.subplot(G[0, :])
ax2 = plt.subplot(G[1, 0])

# Reachability plot
colors = ['g.', 'r.', 'b.', 'y.', 'c.']
for klass, color in zip(range(0, 5), colors):
    Xk = space[labels == klass]
    Rk = reachability[labels == klass]
    ax1.plot(Xk, Rk, color, alpha=0.3)
ax1.plot(space[labels == -1], reachability[labels == -1], 'k.', alpha=0.3)
#ax1.plot(space, np.full_like(space, 2., dtype=float), 'k-', alpha=0.5)
#ax1.plot(space, np.full_like(space, 0.5, dtype=float), 'k-.', alpha=0.5)
ax1.set_ylabel('Reachability (epsilon distance)')
ax1.set_title('Reachability Plot')

# OPTICS
colors = ['g.', 'r.', 'b.', 'y.', 'c.']
for klass, color in zip(range(0, 5), colors):
    Xk = X_hsall[clust.labels_ == klass]
    ax2.plot(Xk[:, 0], Xk[:, 1], color, alpha=0.3)

ax2.plot(X_hsall[clust.labels_ == -1, 0], X_hsall[clust.labels_ == -1, 1], 'k+', alpha=0.1)

ax2.set_title('Automatic Clustering\nOPTICS')